In [ ]:
!pip install python-multipart==0.0.12
!pip install openai requests pathlib
!pip install pypdf2
!pip install pinecone-client openai
!pip install --upgrade llama-index
!pip install llama-index-vector-stores-pinecone
!pip install llama-index-readers-papers
!pip install llama-index-readers-semanticscholar
!pip install llama-index-llms-openai
!pip install langchain langchain-openai
!pip install PyPDF2
!pip install pydantic
!pip install llama-index-readers-pdb
!pip install biopython
!pip install gradio_molecule3d
!pip install opentelemetry-api
!pip install opentelemetry-sdk
!pip install opentelemetry-instrumentation-openai
!pip install phoenix-ai
!pip install "arize-phoenix>=4.29.0" openinference-instrumentation-openai
!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp-proto-http
!pip install openinference-instrumentation-openai openinference-instrumentation-llama-index
!pip install arize-phoenix-otel
!pip uninstall tenacity -y
!pip install tenacity==8.2.3
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing instal

In [ ]:
%%writefile config.py

import os
from pathlib import Path
from dotenv import load_dotenv
import logging
from typing import Dict, Optional

class EnvironmentManager:
    """Manages environment variables and configuration for the protein design application."""

    required_vars = {
        'PINECONE_API_KEY': 'Your Pinecone API key',
        'OPENAI_API_KEY': 'Your OpenAI API key',
        'PHOENIX_API_KEY': 'Your Phoenix API key',
        'NVIDIA_API_KEY': 'Your NVIDIA API key'
    }

    def __init__(self, env_path: Optional[str] = None):
        self.logger = logging.getLogger(__name__)
        self.env_path = env_path or '.env'
        self.env_vars: Dict[str, str] = {}

    def setup_environment(self) -> bool:
        try:
            if not os.path.exists(self.env_path):
                self._create_env_file()
                self.logger.info(f"Created new .env file at {self.env_path}")
                return False

            load_dotenv(self.env_path)

            missing_vars = []
            for var_name, description in self.required_vars.items():
                value = os.getenv(var_name)
                if not value:
                    missing_vars.append(var_name)
                else:
                    self.env_vars[var_name] = value

            if missing_vars:
                self.logger.warning(f"Missing required environment variables: {', '.join(missing_vars)}")
                return False

            self.logger.info("Environment setup completed successfully")
            return True

        except Exception as e:
            self.logger.error(f"Error setting up environment: {str(e)}")
            return False

    def _create_env_file(self) -> None:
        template = "\n".join([f"{var}='{desc}'" for var, desc in self.required_vars.items()])

        with open(self.env_path, 'w') as f:
            f.write(template)

    def get_var(self, var_name: str) -> Optional[str]:
        return self.env_vars.get(var_name)

    @property
    def is_configured(self) -> bool:
        return all(os.getenv(var) for var in self.required_vars)

    def update_var(self, var_name: str, value: str) -> bool:
        try:
            if var_name not in self.required_vars:
                self.logger.error(f"Invalid environment variable: {var_name}")
                return False

            if os.path.exists(self.env_path):
                with open(self.env_path, 'r') as f:
                    lines = f.readlines()
            else:
                lines = []

            var_found = False
            for i, line in enumerate(lines):
                if line.startswith(f"{var_name}="):
                    lines[i] = f"{var_name}='{value}'\n"
                    var_found = True
                    break

            if not var_found:
                lines.append(f"{var_name}='{value}'\n")

            with open(self.env_path, 'w') as f:
                f.writelines(lines)

            os.environ[var_name] = value
            self.env_vars[var_name] = value

            self.logger.info(f"Successfully updated {var_name}")
            return True

        except Exception as e:
            self.logger.error(f"Error updating environment variable: {str(e)}")
            return False

Writing config.py


In [ ]:
from config import EnvironmentManager

# Initialize environment manager
env_manager = EnvironmentManager()

# Set up environment (this will create .env file if it doesn't exist)
if not env_manager.setup_environment():
    print("Please update your .env file with your API keys")
else:
    print("Environment configured successfully!")

# Update with your actual API keys
env_manager.update_var('PINECONE_API_KEY', 'your_pinecone_key_here')
env_manager.update_var('OPENAI_API_KEY', 'your_openai_key_here')
env_manager.update_var('PHOENIX_API_KEY', 'your_phoenix_key_here')
env_manager.update_var('NVIDIA_API_KEY', 'your_nvidia_key_here')

In [ ]:
import os
import openai
import gradio as gr
from openai import OpenAI
import logging
import traceback
import io
import time
import requests
import json
from typing import List, Optional, Tuple, Union, Dict, Any
from pydantic import BaseModel, Field
import pinecone
from llama_index.core import VectorStoreIndex, StorageContext, Document, Settings
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone
import requests
import json
from pathlib import Path
from llama_index.readers.papers import ArxivReader, PubmedReader
from llama_index.readers.pdb import PdbAbstractReader
from Bio import PDB
from gradio_molecule3d import Molecule3D
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
import tempfile
from llama_index.llms.openai import OpenAI as LlamaOpenAI
import base64
import tempfile
from config import EnvironmentManager
from pathlib import Path
from tenacity import retry, stop_after_attempt, wait_exponential
from contextlib import nullcontext
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.resources import Resource
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

# Initialize environment
env_manager = EnvironmentManager()
if not env_manager.setup_environment():
    raise RuntimeError("Environment not properly configured")

# Use the environment variables
PINECONE_API_KEY = env_manager.get_var('PINECONE_API_KEY')
OPENAI_API_KEY = env_manager.get_var('OPENAI_API_KEY')
PHOENIX_API_KEY = env_manager.get_var('PHOENIX_API_KEY')
NVIDIA_API_KEY = env_manager.get_var('NVIDIA_API_KEY')

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class AlphaFold2Output(BaseModel):
    analysis: str = Field(description="Analysis of AlphaFold2 parameters")
    code: str = Field(description="Generated Python code for running AlphaFold2")

# Add the GROMACS-specific Protocol class with your existing classes
class Protocol(BaseModel):
    steps: List[str] = Field(description="Steps for the GROMACS simulation protocol using NVIDIA tools")

# 2. Proper OpenTelemetry initialization and error handling
class TracingSetup:
    def __init__(
        self,
        project_name: str = "protein-design-assistant",
        phoenix_api_key: Optional[str] = None,
    ):
        self.project_name = project_name
        self.phoenix_api_key = phoenix_api_key or os.getenv("PHOENIX_API_KEY")
        self.tracer_provider = None
        self.tracer = None

    def initialize_tracing(self) -> None:
        """Initialize OpenTelemetry tracing with Phoenix integration."""
        try:
            if not self.phoenix_api_key:
                logger.warning("Phoenix API key not found. Tracing will be disabled.")
                return None

            # Set the tracer provider as the global default
            resource = Resource.create({
                "service.name": self.project_name,
                "deployment.environment": "colab"
            })

            self.tracer_provider = TracerProvider(resource=resource)
            trace.set_tracer_provider(self.tracer_provider)

            # Configure Phoenix exporter
            exporter = OTLPSpanExporter(
                endpoint="https://app.phoenix.arize.com/v1/traces",
                headers={"api_key": self.phoenix_api_key}
            )

            # Add BatchSpanProcessor with the Phoenix exporter
            processor = BatchSpanProcessor(exporter)
            self.tracer_provider.add_span_processor(processor)

            # Initialize instrumentors only if tracer provider is set
            if self.tracer_provider:
                OpenAIInstrumentor().instrument(tracer_provider=self.tracer_provider)
                LlamaIndexInstrumentor().instrument(tracer_provider=self.tracer_provider)

            # Create tracer
            self.tracer = trace.get_tracer(self.project_name)
            logger.info(f"OpenTelemetry tracing initialized for project: {self.project_name}")

        except Exception as e:
            logger.error(f"Failed to initialize tracing: {str(e)}")
            self.tracer_provider = None
            self.tracer = None

    def start_span(self, name: str):
        """Helper method to start a new span with error handling"""
        try:
            if self.tracer is None:
                return nullcontext()
            return self.tracer.start_span(name)
        except Exception as e:
            logger.error(f"Error starting span: {str(e)}")
            return nullcontext()

    def start_as_current_span(self, name: str):
        """Helper method to start a new span as current with error handling"""
        try:
            if self.tracer is None:
                return nullcontext()
            return self.tracer.start_as_current_span(name)
        except Exception as e:
            logger.error(f"Error starting current span: {str(e)}")
            return nullcontext()

class AudioResponseHandler:
    def __init__(self, openai_client: OpenAI):
        self.client = openai_client
        self.voices = ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]
        self.current_voice = "alloy"

    def text_to_speech(self, text: str) -> str:
        """Convert text to speech and return the path to the audio file"""
        if not text.strip():
            return None

        try:
            # Create a temporary file with .mp3 extension
            with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as temp_file:
                response = self.client.audio.speech.create(
                    model="tts-1",
                    voice=self.current_voice,
                    input=text
                )
                response.stream_to_file(temp_file.name)
                return temp_file.name

        except Exception as e:
            logger.error(f"Error in text to speech conversion: {str(e)}")
            return None

    def set_voice(self, voice: str):
        """Set the TTS voice"""
        if voice in self.voices:
            self.current_voice = voice

# AlphaFold2 specific state management
class AlphaFold2State:
    def __init__(self):
        self.parameters = None
        self.code = None

af2_state = AlphaFold2State()

class PineconeManager:
    def __init__(self, api_key: str = None, index_name: str = "llama-index-nvidia"):
        self.api_key = api_key or os.getenv("PINECONE_API_KEY")
        self.index_name = index_name
        self.pc = None
        self.vector_store_index = None
        self.query_engine = None

    def initialize(self) -> None:
        """Initialize Pinecone connection and index"""
        try:
            # Initialize Pinecone
            self.pc = Pinecone(api_key=self.api_key)

            # Check if index exists, create if it doesn't
            if self.index_name not in self.pc.list_indexes().names():
                self.pc.create_index(
                    name=self.index_name,
                    dimension=1536,  # OpenAI embedding dimension
                    metric="cosine",
                    spec={'pod_type': 'p1'}
                )
                logger.info(f"Created new Pinecone index: {self.index_name}")

            # Get the index
            pinecone_index = self.pc.Index(self.index_name)

            # Set up vector store
            vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
            storage_context = StorageContext.from_defaults(vector_store=vector_store)

            # Configure Llama-index settings
            Settings.chunk_size = 1024
            Settings.chunk_overlap = 20

            # Create vector store index
            self.vector_store_index = VectorStoreIndex.from_vector_store(
                vector_store=vector_store,
                storage_context=storage_context
            )

            # Initialize query engine
            self.query_engine = self.vector_store_index.as_query_engine()

            logger.info("Pinecone initialization completed successfully")

        except Exception as e:
            logger.error(f"Error initializing Pinecone: {str(e)}")
            raise

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def query(self, query_text: str) -> Optional[str]:
        """
        Query the Pinecone index with retry logic

        Args:
            query_text: The text to query

        Returns:
            Query response or None if failed
        """
        try:
            if not self.query_engine:
                raise ValueError("Query engine not initialized. Call initialize() first.")

            response = self.query_engine.query(query_text)
            return str(response.response)

        except Exception as e:
            logger.error(f"Error querying Pinecone: {str(e)}")
            raise

    def add_documents(self, documents: Union[Document, List[Document]]) -> None:
        """
        Add documents to the Pinecone index

        Args:
            documents: Single document or list of documents to add
        """
        try:
            if not self.vector_store_index:
                raise ValueError("Vector store index not initialized. Call initialize() first.")

            if isinstance(documents, Document):
                documents = [documents]

            self.vector_store_index.insert_nodes(documents)
            logger.info(f"Successfully added {len(documents)} documents to Pinecone index")

        except Exception as e:
            logger.error(f"Error adding documents to Pinecone: {str(e)}")
            raise

# Add a monitor to track the chat system's health
class ChatSystemMonitor:
    def __init__(self):
        self.total_calls = 0
        self.successful_calls = 0
        self.failed_calls = 0
        self.last_error = None
        self.start_time = time.time()

    def record_success(self):
        self.total_calls += 1
        self.successful_calls += 1

    def record_failure(self, error):
        self.total_calls += 1
        self.failed_calls += 1
        self.last_error = error

    def get_status(self):
        uptime = time.time() - self.start_time
        success_rate = (self.successful_calls / self.total_calls * 100) if self.total_calls > 0 else 0
        return {
            "uptime": uptime,
            "total_calls": self.total_calls,
            "success_rate": success_rate,
            "last_error": str(self.last_error) if self.last_error else None
        }

# Initialize the monitor
chat_monitor = ChatSystemMonitor()

# Initialize tracing setup
tracing = TracingSetup(phoenix_api_key=os.getenv("PHOENIX_API_KEY"))

# Initialize OpenAI client and Pinecone
openai_client = OpenAI()
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Initialize DeepSeek client
deepseek_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.getenv("NVIDIA_API_KEY")
)

# Add this after your other client initializations
llm = LlamaOpenAI(
    model="gpt-4o",  # or whatever model you prefer
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0,
)

def setup_pinecone_index():
    """
    Initialize and configure Pinecone index with proper error handling
    """
    try:
        logger.info("Starting Pinecone index setup")

        # Validate environment variables
        pinecone_key = os.getenv("PINECONE_API_KEY")
        if not pinecone_key:
            raise ValueError("PINECONE_API_KEY environment variable is not set")

        # Initialize Pinecone client with retry logic
        @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
        def init_pinecone():
            return Pinecone(api_key=pinecone_key)

        pc = init_pinecone()
        index_name = "llama-index-nvidia"

        # Check if index exists, create if it doesn't
        existing_indexes = pc.list_indexes().names()
        if index_name not in existing_indexes:
            logger.info(f"Creating new Pinecone index: {index_name}")
            pc.create_index(
                name=index_name,
                dimension=1536,  # OpenAI embedding dimension
                metric="cosine",
                spec={'pod_type': 'p1'}
            )

        # Get the index
        pinecone_index = pc.Index(index_name)

        # Set up vector store
        vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        # Configure Llama-index settings
        Settings.chunk_size = 1024
        Settings.chunk_overlap = 20

        # Create and return vector store index
        vector_store_index = VectorStoreIndex.from_vector_store(
            vector_store=vector_store,
            storage_context=storage_context
        )

        logger.info("Successfully initialized Pinecone index")
        return vector_store_index

    except Exception as e:
        logger.error(f"Failed to set up Pinecone index: {str(e)}")
        logger.error(f"Full traceback: {traceback.format_exc()}")
        raise RuntimeError(f"Pinecone setup failed: {str(e)}")

# Usage example:
def setup_pinecone_and_query_engine():
    try:
        pinecone_manager = PineconeManager()
        pinecone_manager.initialize()
        return pinecone_manager.query_engine
    except Exception as e:
        logger.error(f"Error setting up Pinecone: {str(e)}")
        raise

# Update your existing query engine initialization
query_engine = setup_pinecone_and_query_engine()

def chat_with_gpt4o(message, chat_history, history_state):
    """
    Enhanced NIMs chat assistant with proper flow control and error handling
    """
    try:
        if history_state is None:
            history_state = []

        logger.info("Starting chat interaction")

        # Phase 1: Query Preparation
        try:
            logger.info("Preparing query and context")
            # Create a synchronous query to prevent stalling
            query_result = query_engine.query(
                message,
                response_mode="compact",  # Use compact mode for faster processing
                timeout=30  # Add timeout to prevent hanging
            )
            relevant_context = str(query_result.response)
            logger.info("Context retrieved successfully")
        except Exception as e:
            logger.error(f"Error in query phase: {str(e)}")
            relevant_context = "Unable to retrieve context. Proceeding with basic response."

        # Phase 2: Chat Completion
        try:
            logger.info("Initiating chat completion")
            system_message = {
                "role": "system",
                "content": f"""You are an AI assistant specializing in protein design and NVIDIA technologies.
                Use this relevant information to inform your response: {relevant_context}"""
            }

            messages = [system_message] + history_state + [{"role": "user", "content": message}]

            response = openai_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                max_tokens=16384,
                temperature=0,
                stream=False,
                timeout=60  # Add timeout for chat completion
            )

            response_text = response.choices[0].message.content
            logger.info("Chat completion successful")
        except Exception as e:
            logger.error(f"Error in chat completion: {str(e)}")
            return message, chat_history, history_state, None

        # Phase 3: Audio Generation
        try:
            logger.info("Starting audio generation")
            audio_handler = AudioResponseHandler(openai_client)
            audio_file_path = audio_handler.text_to_speech(response_text)

            if not audio_file_path:
                logger.warning("Audio generation returned no file path")
                raise ValueError("Audio generation failed")

            logger.info(f"Audio generated successfully: {audio_file_path}")
        except Exception as e:
            logger.error(f"Error in audio generation: {str(e)}")
            audio_file_path = None
            response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

        # Update conversation state
        chat_history = chat_history + [(message, response_text)]
        history_state.append({"role": "assistant", "content": response_text})

        logger.info("Chat interaction completed successfully")
        return message, chat_history, history_state, audio_file_path

    except Exception as e:
        logger.error(f"Critical error in chat system: {str(e)}")
        error_message = "I apologize, but I encountered an unexpected error. Please try again."
        chat_history = chat_history + [(message, error_message)]
        return message, chat_history, history_state, None


def fetch_papers(query: str, source: str, max_results: int = 10) -> List[Document]:
    """
    Fetch papers from specified source with improved error handling.
    """
    try:
        with tracing.start_as_current_span("fetch_papers") as span:
            span.set_attribute("query", query)
            span.set_attribute("source", source)
            span.set_attribute("max_results", max_results)

            documents = []
            if source == "arxiv":
                reader = ArxivReader()
                documents = reader.load_data(search_query=query, max_results=max_results)
            elif source == "pubmed":
                reader = PubmedReader()
                documents = reader.load_data(search_query=query, max_results=max_results)

            span.set_attribute("documents_found", len(documents))
            return documents

    except Exception as e:
        logger.error(f"Error fetching papers: {str(e)}")
        return []

def search_and_summarize_papers(query: str, source: str) -> tuple[str, str]:
    """
    Search and summarize scientific papers with proper error handling, token limits,
    tracing spans, system messages for the chatbot, and audio output generation.
    Returns both text summary and audio file path.
    """
    try:
        with tracing.start_as_current_span("search_and_summarize_papers") as span:
            span.set_attribute("query", query)
            span.set_attribute("source", source)

            # Input validation
            if not query.strip():
                return "Please enter a search query.", None

            if source not in ["arxiv", "pubmed"]:
                return "Invalid source. Please select either 'arxiv' or 'pubmed'.", None

            # Fetch papers with tracing
            with tracing.start_as_current_span("fetch_papers") as fetch_span:
                papers = fetch_papers(query, source, max_results=1)
                if not papers:
                    return "No papers found matching the query.", None
                fetch_span.set_attribute("papers_found", len(papers))

            # Create a concise summary prompt with system message
            paper = papers[0]
            title = paper.metadata.get('title', 'N/A')[:200]
            abstract = paper.text[:500] + "..." if len(paper.text) > 500 else paper.text

            system_message = {
                "role": "system",
                "content": """You are an AI assistant specializing in scientific literature analysis
                and NVIDIA technologies. Analyze the given paper and explain:
                1. The main findings and their significance
                2. How the research relates to NVIDIA technologies and GPU computing
                3. Potential applications and impact on computational biology
                4. Technical implementation considerations
                Be concise but thorough in your analysis."""
            }

            summary_prompt = f"""Analyze this scientific paper in the context of NVIDIA technologies:

            Title: {title}
            Abstract: {abstract}

            Focus on:
            1. Core findings and methodology
            2. Relevance to GPU computing and NVIDIA platforms
            3. Potential applications in computational biology
            4. Implementation considerations
            """

            # Generate summary with tracing and error handling
            with tracing.start_as_current_span("query_engine_summary") as query_span:
                try:
                    response = openai_client.chat.completions.create(
                        model="gpt-4o",
                        messages=[
                            system_message,
                            {"role": "user", "content": summary_prompt}
                        ],
                        max_tokens=1024,
                        temperature=0.7
                    )
                    summary = response.choices[0].message.content
                    query_span.set_attribute("summary_length", len(summary))

                    # Generate audio for the summary
                    with tracing.start_as_current_span("generate_audio_summary") as audio_span:
                        try:
                            audio_handler = AudioResponseHandler(openai_client)
                            audio_file_path = audio_handler.text_to_speech(summary)
                            audio_span.set_attribute("audio_generated", bool(audio_file_path))
                            logger.info("Generated audio summary successfully")
                        except Exception as audio_error:
                            logger.error(f"Error generating audio summary: {str(audio_error)}")
                            audio_file_path = None

                    return summary, audio_file_path

                except Exception as query_error:
                    error_msg = f"Error generating summary: {str(query_error)}"
                    query_span.set_attribute("error", error_msg)
                    logger.error(error_msg)
                    raise

    except Exception as e:
        error_msg = f"Error in paper search: {str(e)}"
        logger.error(error_msg)
        if span:  # Check if span exists in case error occurred before span creation
            span.set_attribute("error", error_msg)
        return error_msg, None

# Function to fetch PDB abstract
def fetch_pdb_abstract(pdb_id):
    try:
        loader = PdbAbstractReader()
        documents = loader.load_data([pdb_id])
        return documents[0].text if documents else "No abstract found for the given PDB ID."
    except Exception as e:
        logger.error(f"Error fetching PDB abstract: {e}")
        return f"An error occurred while fetching the PDB abstract: {str(e)}"

# Function to download PDB file
def download_pdb(pdb_id):
    try:
        pdb_list = PDB.PDBList()
        file_path = pdb_list.retrieve_pdb_file(
            pdb_id, pdir="./", file_format="pdb"
        )

        # Rename the file to ensure .pdb extension
        new_file_path = f"{pdb_id}.pdb"
        os.rename(file_path, new_file_path)

        return f"PDB file for {pdb_id} downloaded successfully to {new_file_path}"
    except Exception as e:
        logger.error(f"Error downloading PDB file: {str(e)}")
        return f"Error downloading PDB file: {str(e)}"

def process_pdb_file(file):
    try:
        if hasattr(file, 'read'):
            content = file.read()
            if isinstance(content, bytes):
                content = content.decode('utf-8')
        elif hasattr(file, 'name'):
            with open(file.name, 'r') as f:
                content = f.read()
        else:
            raise ValueError("Unsupported file object type")

        return content
    except Exception as e:
        logger.error(f"Error processing PDB file: {str(e)}")
        raise gr.Error(f"Error processing PDB file: {str(e)}")

def get_rfdiffusion_parameters(pdb_content):
    """
    Generate simplified, reliable parameters for RFdiffusion based on PDB content,
    now with voice explanation.
    """
    try:
        # Create a simplified analysis query that focuses on core requirements
        query = f"""
        Analyze this PDB content and suggest basic parameters for RFdiffusion that will work reliably.
        Follow this exact format in your response:

        PARAMETERS:
        1. contigs: [value]
        2. hotspot_res: [value]
        3. diffusion_steps: [value]

        EXPLANATION:
        Provide a clear, conversational explanation of why these parameters were chosen and how they will affect the protein design process.
        Keep the explanation natural and suitable for voice output.

        Base the analysis on this PDB structure:
        {pdb_content[:500]}...
        """

        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": """You are an expert in RFdiffusion parameter optimization.
                Always ensure contigs parameter has both target specification AND binder length range after /0.
                Example: 'A20-60/0 50-100' NOT just 'A20-60/0'"""},
                {"role": "user", "content": query}
            ]
        )

        # Split response into parameters and explanation
        response_text = response.choices[0].message.content
        parts = response_text.split("EXPLANATION:", 1)
        parameters = parts[0].strip()
        explanation = parts[1].strip() if len(parts) > 1 else "No explanation provided."

        # Generate audio for the explanation
        audio_handler = AudioResponseHandler(openai_client)
        audio_file_path = audio_handler.text_to_speech(explanation)

        return parameters, explanation, audio_file_path
    except Exception as e:
        logger.error(f"Error in GPT-4o analysis: {str(e)}")
        raise gr.Error(f"Error in parameter analysis: {str(e)}")

def generate_rfdiffusion_code(pdb_content, parameters):
    """
    Generate reliable RFdiffusion code using simplified parameters.
    """
    try:
        # Parse only essential parameters
        params = {}
        for line in parameters.split('\n'):
            if ':' in line:
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()

                if key == 'contigs':
                    # Ensure contig format includes both target and binder specifications
                    value = value.strip('"\' ')
                    if '/0' not in value:
                        # If missing proper format, use safe default
                        value = 'A20-60/0 50-100'
                    elif len(value.split('/0')) != 2:
                        # If no binder length specified after /0, add default
                        value = f"{value.split('/0')[0]}/0 50-100"
                    params[key] = value
                elif key == 'hotspot_res':
                    # Convert string representation to list
                    value = eval(value) if '[' in value else [x.strip() for x in value.split(',')]
                    params[key] = value
                elif key == 'diffusion_steps':
                    # Ensure minimum of 15 steps
                    try:
                        steps = max(15, int(value))
                        params[key] = steps
                    except ValueError:
                        params[key] = 15

        code = f"""#!/usr/bin/env python3

import requests
import os
import json
from pathlib import Path
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_api_key():
    return os.getenv("NVCF_RUN_KEY") or input("Paste the Run Key: ")

def get_pdb_content():
    return \"\"\"
{pdb_content}
\"\"\"

def run_rfdiffusion(api_key, input_pdb):
    try:
        url = "https://health.api.nvidia.com/v1/biology/ipd/rfdiffusion/generate"
        headers = {{
            "Authorization": f"Bearer {{api_key}}",
            "Accept": "application/json"
        }}

        # Core parameters only - keeping it simple and reliable
        payload = {{
            "input_pdb": input_pdb,
            "contigs": "{params.get('contigs', 'A20-60/0 50-100')}",  # Default includes both target and binder specs
            "hotspot_res": {params.get('hotspot_res', ['A50', 'A51', 'A52'])},
            "diffusion_steps": {params.get('diffusion_steps', 15)}
        }}

        logger.info(f"Sending request with payload: {{payload}}")
        response = requests.post(url=url, headers=headers, json=payload)
        response.raise_for_status()
        return response

    except requests.exceptions.RequestException as e:
        logger.error(f"API request failed: {{str(e)}}")
        if hasattr(e.response, 'text'):
            logger.error(f"Response content: {{e.response.text}}")
        raise

def save_output(response, output_file="output.pdb"):
    try:
        logger.info(f"Processing response to save to {{output_file}}")
        response_json = response.json()

        if "output_pdb" not in response_json:
            logger.error(f"Expected 'output_pdb' in response, got: {{list(response_json.keys())}}")
            raise KeyError("Response does not contain 'output_pdb' key")

        Path(output_file).write_text(response_json["output_pdb"])
        logger.info(f"Successfully saved output to {{output_file}}")

    except Exception as e:
        logger.error(f"Error saving output: {{str(e)}}")
        raise

def main():
    try:
        api_key = get_api_key()
        input_pdb = get_pdb_content()
        response = run_rfdiffusion(api_key, input_pdb)
        save_output(response)
        logger.info("RFdiffusion process completed successfully")

    except Exception as e:
        logger.error(f"Error in main execution: {{str(e)}}")
        raise

if __name__ == "__main__":
    main()
"""
        return code
    except Exception as e:
        logger.error(f"Error generating RFdiffusion code: {str(e)}")
        raise gr.Error(f"Error generating code: {str(e)}")

def execute_rfdiffusion_script(api_key: str, pdb_content: str, parameters: str) -> tuple:
    """
    Execute the RFdiffusion script with enhanced error handling and simplified parameters.
    """
    try:
        if not api_key or not api_key.strip():
            raise ValueError("API key is required")

        # Parse only essential parameters
        params = {
            "contigs": "A20-60/0 50-100",  # Default includes both target and binder specs
            "hotspot_res": ["A50", "A51", "A52"],
            "diffusion_steps": 15
        }

        for line in parameters.split('\n'):
            if ':' in line and not line.startswith('-'):
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()

                if key == 'contigs':
                    value = value.strip('"\' ')
                    if '/0' not in value:
                        value = 'A20-60/0 50-100'
                    elif len(value.split('/0')) != 2:
                        value = f"{value.split('/0')[0]}/0 50-100"
                    params[key] = value
                elif key == 'hotspot_res':
                    if isinstance(value, str):
                        if '[' in value:
                            params[key] = eval(value)
                        else:
                            params[key] = [x.strip() for x in value.split(',')]
                elif key == 'diffusion_steps':
                    try:
                        params[key] = max(15, int(value))
                    except ValueError:
                        params[key] = 15

        # Make API request
        url = "https://health.api.nvidia.com/v1/biology/ipd/rfdiffusion/generate"
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Accept": "application/json"
        }

        payload = {
            "input_pdb": pdb_content,
            "contigs": params["contigs"],
            "hotspot_res": params["hotspot_res"],
            "diffusion_steps": params["diffusion_steps"]
        }

        logger.info(f"Making RFdiffusion API request with parameters: {params}")
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()

        result = response.json()

        if "output_pdb" in result:
            output_pdb_content = result["output_pdb"]
            output_file = Path("rfdiffusion_output.pdb")
            output_file.write_text(output_pdb_content)

            status_message = (
                f"Success! Generated structure saved to: {output_file}\n"
                f"Parameters used:\n"
                f"- Contigs: {params['contigs']}\n"
                f"- Hotspot residues: {params['hotspot_res']}\n"
                f"- Diffusion steps: {params['diffusion_steps']}"
            )
            return status_message, output_pdb_content, True
        else:
            return f"Error: Unexpected response format\n{response.text}", None, False

    except requests.exceptions.RequestException as e:
        logger.error(f"API Request Error: {str(e)}")
        if hasattr(e.response, 'text'):
            logger.error(f"Response content: {e.response.text}")
        return f"API Request Error: {str(e)}", None, False
    except Exception as e:
        logger.error(f"Error executing RFdiffusion: {str(e)}")
        return f"Error executing RFdiffusion: {str(e)}", None, False

def rfdiffusion_app(file):
    try:
        if file is None:
            raise gr.Error("No file uploaded. Please upload a PDB file.")

        pdb_content = process_pdb_file(file)
        parameters, explanation, audio_file = get_rfdiffusion_parameters(pdb_content)
        code = generate_rfdiffusion_code(pdb_content, parameters)

        return parameters, code, explanation, audio_file
    except gr.Error as e:
        raise
    except Exception as e:
        logger.error(f"Unexpected error in rfdiffusion_app: {str(e)}")
        raise gr.Error(f"An unexpected error occurred: {str(e)}")

def get_proteinmpnn_parameters(pdb_content):
    try:
        query = f"""
        Analyze the following PDB content and suggest appropriate parameters for running ProteinMPNN:

        {pdb_content[:500]}...

        Provide a clear, conversational explanation of:
        1. ca_only setting and why it's appropriate
        2. use_soluble_model choice based on the structure
        3. num_seq_per_target recommendation
        4. sampling_temp selection

        Format your response with clear parameter values first, followed by a natural explanation suitable for voice synthesis.
        """

        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert in ProteinMPNN modeling. Provide parameters followed by a clear, conversational explanation."},
                {"role": "user", "content": query}
            ]
        )

        # Split response into parameters and explanation
        response_text = response.choices[0].message.content
        parts = response_text.split("\n\n", 1)
        parameters = parts[0]
        explanation = parts[1] if len(parts) > 1 else "No explanation provided."

        # Generate audio for the explanation
        audio_handler = AudioResponseHandler(openai_client)
        audio_file_path = audio_handler.text_to_speech(explanation)

        return parameters, explanation, audio_file_path
    except Exception as e:
        logger.error(f"Error in GPT-4o analysis: {str(e)}")
        raise gr.Error(f"Error in GPT-4o analysis: {str(e)}")

def generate_proteinmpnn_code(pdb_content, parameters):
    try:
        # Parse parameters from the analysis
        params = {}
        explanation = []
        current_param = None

        for line in parameters.split('\n'):
            line = line.strip()
            if not line:
                continue

            # Check if this is a parameter line
            if ': ' in line and not line.startswith('-'):
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()

                # Convert string representations to appropriate Python types
                if key == 'sampling_temp':
                    try:
                        temp = float(value)
                        params[key] = [temp]
                    except ValueError:
                        params[key] = [0.1]
                elif key in ['ca_only', 'use_soluble_model']:
                    params[key] = value.lower() == 'true'
                elif key == 'num_seq_per_target':
                    try:
                        params[key] = int(value)
                    except ValueError:
                        params[key] = 1

                current_param = key
            elif line.startswith('-') and current_param:
                explanation.append((current_param, line[1:].strip()))

        code = f"""#!/usr/bin/env python3

import requests
import os
import json
from pathlib import Path

def get_api_key():
    return os.getenv("NVCF_RUN_KEY") or input("Paste the Run Key: ")

def get_pdb_content():
    return \"\"\"
{pdb_content}
\"\"\"

def run_proteinmpnn(api_key, input_pdb, ca_only, use_soluble_model, num_seq_per_target, sampling_temp):
    url = "https://health.api.nvidia.com/v1/biology/ipd/proteinmpnn/predict"
    headers = {{"Authorization": f"Bearer {{api_key}}", "Content-Type": "application/json"}}
    payload = {{
        "input_pdb": input_pdb,
        "ca_only": ca_only,
        "use_soluble_model": use_soluble_model,
        "num_seq_per_target": num_seq_per_target,
        "sampling_temp": sampling_temp
    }}

    response = requests.post(url=url, headers=headers, json=payload)
    if response.status_code != 200:
        raise Exception(f"API request failed with status {{response.status_code}}: {{response.text}}")
    return response

def save_output(response, output_file="output.fasta"):
    try:
        response_data = json.loads(response.text)
        if "mfasta" not in response_data:
            raise KeyError("Response does not contain 'mfasta' key")
        print(response, f"Saving to {{output_file}}:\\n", response.text[:200], "...")
        Path(output_file).write_text(response_data["mfasta"])
    except json.JSONDecodeError as e:
        raise Exception(f"Failed to parse response JSON: {{str(e)}}")
    except KeyError as e:
        raise Exception(f"Invalid response format: {{str(e)}}")

def main():
    # Parameters explanation from structure analysis:
{chr(10).join(f'    # {param}: {exp}' for param, exp in explanation)}

    api_key = get_api_key()
    input_pdb = get_pdb_content()
    ca_only = {str(params.get('ca_only', False))}
    use_soluble_model = {str(params.get('use_soluble_model', False))}
    num_seq_per_target = {params.get('num_seq_per_target', 1)}
    sampling_temp = {params.get('sampling_temp', [0.1])}

    try:
        response = run_proteinmpnn(api_key, input_pdb, ca_only, use_soluble_model, num_seq_per_target, sampling_temp)
        save_output(response)
    except Exception as e:
        print(f"Error: {{str(e)}}")

if __name__ == "__main__":
    main()
"""
        return code
    except Exception as e:
        logger.error(f"Error generating ProteinMPNN code: {{str(e)}}")
        raise gr.Error(f"Error generating ProteinMPNN code: {{str(e)}}")

# Add this function to execute ProteinMPNN script
def execute_proteinmpnn_script(api_key: str, pdb_content: str, parameters: str) -> tuple:
    """
    Execute the ProteinMPNN script and return both results and FASTA content.
    Returns tuple of (status_message, fasta_content, success_bool)
    """
    try:
        if not api_key or not api_key.strip():
            raise ValueError("API key is required")

        # Parse parameters from the analysis
        params = {}
        for line in parameters.split('\n'):
            if ':' in line and not line.startswith('-'):
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()

                # Convert string representations to appropriate Python types
                if key == 'sampling_temp':
                    try:
                        temp = float(value)
                        params[key] = [temp]
                    except ValueError:
                        params[key] = [0.1]
                elif key in ['ca_only', 'use_soluble_model']:
                    params[key] = value.lower() == 'true'
                elif key == 'num_seq_per_target':
                    try:
                        params[key] = int(value)
                    except ValueError:
                        params[key] = 1

        # Make API request
        url = "https://health.api.nvidia.com/v1/biology/ipd/proteinmpnn/predict"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        payload = {
            "input_pdb": pdb_content,
            "ca_only": params.get('ca_only', False),
            "use_soluble_model": params.get('use_soluble_model', False),
            "num_seq_per_target": params.get('num_seq_per_target', 1),
            "sampling_temp": params.get('sampling_temp', [0.1])
        }

        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()

        # Parse the response
        result = response.json()

        if "mfasta" in result:
            fasta_content = result["mfasta"]

            # Save the output FASTA to a temporary file
            output_file = Path("proteinmpnn_output.fasta")
            output_file.write_text(fasta_content)

            status_message = (
                f"Success! Generated sequences saved to: {output_file}\n\n"
                f"Output FASTA summary:\n"
                f"- File size: {len(fasta_content)} bytes\n"
                f"- First few lines:\n{fasta_content[:200]}..."
            )
            return status_message, fasta_content, True
        else:
            return f"Error: Unexpected response format\n{response.text}", None, False

    except requests.exceptions.RequestException as e:
        return f"API Request Error: {str(e)}", None, False
    except Exception as e:
        return f"Error executing ProteinMPNN: {str(e)}", None, False

def proteinmpnn_app(file, api_key: str = None):
    try:
        if file is None:
            raise gr.Error("No file uploaded. Please upload a PDB file.")

        pdb_content = process_pdb_file(file)
        parameters, explanation, audio_file = get_proteinmpnn_parameters(pdb_content)
        code = generate_proteinmpnn_code(pdb_content, parameters)

        # If API key is provided, execute the script
        if api_key:
            status, fasta_content, success = execute_proteinmpnn_script(api_key, pdb_content, parameters)
            return parameters, code, explanation, audio_file, status, fasta_content

        return parameters, code, explanation, audio_file, "", ""

    except gr.Error as e:
        raise
    except Exception as e:
        logger.error(f"Unexpected error in proteinmpnn_app: {str(e)}")
        raise gr.Error(f"An unexpected error occurred: {str(e)}")

# New function for PDB visualization
def visualize_pdb(file):
    if file is None:
        return None
    return file

# New function for Nvidia NIMs chat
def chat_with_nims(message, chat_history, history_state):
    try:
        if history_state is None:
            history_state = []

        # Query the Pinecone database for relevant information
        query_result = query_engine.query(message)
        relevant_info = query_result.response

        # Prepare the system message
        system_message = f"""You are an AI assistant specializing in Nvidia NIMs (NVIDIA AI Foundation models) and protein design research.
        Use the following information to help answer the user's question: {relevant_info}
        If you don't have enough information to answer the question, please say so and offer to help with other aspects of Nvidia NIMs or protein design."""

        history_state.append({"role": "system", "content": system_message})
        history_state.append({"role": "user", "content": message})
        messages = history_state.copy()

        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            max_tokens=16384,
            temperature=0.7,
            stream=True
        )

        # Get the complete response text
        response_text = response.choices[0].message.content

        # Update chat history
        chat_history = chat_history + [(message, response_text)]

        # Generate audio for the complete response
        audio_handler = AudioResponseHandler(openai_client)
        audio_file_path = audio_handler.text_to_speech(response_text)

        for chunk in response:
            if chunk.choices:
                delta = chunk.choices[0].delta
                if delta.content:
                    token = delta.content
                    response_text += token
                    chat_history[-1] = (message, response_text)
                    yield "", chat_history, history_state

        history_state.append({"role": "assistant", "content": response_text})

        return message, chat_history, history_state, audio_file_path

    except Exception as e:
        logger.error(f"Error in Nvidia NIMs chat: {str(e)}")
        yield "", chat_history, history_state

def get_molmim_parameters(smiles):
    try:
        query = f"""
        Analyze the following SMILES string and suggest appropriate parameters for running MolMIM:

        {smiles}

        Provide the following parameters:
        1. algorithm
        2. num_molecules
        3. property_name
        4. minimize
        5. min_similarity
        6. particles
        7. iterations

        Explain your reasoning for each parameter.
        """
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert in molecular generation and optimization using MolMIM."},
                {"role": "user", "content": query}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Error in GPT-4 analysis: {str(e)}")
        raise gr.Error(f"Error in GPT-4 analysis: {str(e)}")

def generate_molmim_code(smiles, parameters):
    try:
        params = {}
        for line in parameters.split('\n'):
            if ':' in line:
                key, value = line.split(':', 1)
                params[key.strip()] = value.strip()

        code = f"""
import requests

def run_molmim(api_key, smiles):
    invoke_url = "https://health.api.nvidia.com/v1/biology/nvidia/molmim/generate"
    headers = {{
        "Authorization": f"Bearer {{api_key}}",
        "Accept": "application/json",
    }}
    payload = {{
        "algorithm": "{params.get('algorithm', 'CMA-ES')}",
        "num_molecules": {params.get('num_molecules', '30')},
        "property_name": "{params.get('property_name', 'QED')}",
        "minimize": {params.get('minimize', 'False')},
        "min_similarity": {params.get('min_similarity', '0.3')},
        "particles": {params.get('particles', '30')},
        "iterations": {params.get('iterations', '10')},
        "smi": "{smiles}"
    }}

    session = requests.Session()
    response = session.post(invoke_url, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()

def main():
    api_key = input("Enter your NVIDIA API key: ")
    smiles = "{smiles}"
    result = run_molmim(api_key, smiles)
    print(result)

if __name__ == "__main__":
    main()
"""
        return code
    except Exception as e:
        logger.error(f"Error generating MolMIM code: {str(e)}")
        raise gr.Error(f"Error generating MolMIM code: {str(e)}")

def molmim_app(smiles):
    try:
        if not smiles:
            raise gr.Error("Please enter a SMILES string.")

        parameters = get_molmim_parameters(smiles)
        code = generate_molmim_code(smiles, parameters)

        return parameters, code
    except gr.Error as e:
        raise
    except Exception as e:
        logger.error(f"Unexpected error in molmim_app: {str(e)}")
        raise gr.Error(f"An unexpected error occurred: {str(e)}")

# Add these functions with your other function definitions
def analyze_sequence_with_gpt4(query_engine, sequence):
    try:
        query = f"""
        Analyze the following amino acid sequence and suggest optimal parameters for running AlphaFold2 on the NVIDIA GPU Cloud (NGC):

        Sequence: {sequence[:100]}...

        Provide a clear and concise explanation of the recommended AlphaFold2 parameters for this sequence, focusing on:
        1. algorithm
        2. e_value
        3. iterations
        4. databases
        5. relax_prediction
        6. structure_model_preset
        7. structure_models_to_relax
        8. num_predictions_per_model
        9. max_msa_sequences
        10. template_searcher

        Explain why each parameter is recommended for this specific sequence.
        """
        with tracing.start_as_current_span("analyze_sequence_with_gpt4") as span:
            span.set_attribute("sequence_length", len(sequence))
            response = query_engine.query(query)
            logger.info("Successfully analyzed sequence with GPT-4")
            return str(response)
    except Exception as e:
        logger.error(f"Error in GPT-4 analysis: {str(e)}")
        raise

def generate_alphafold2_code(sequence, analysis):
    try:
        with tracing.start_as_current_span("generate_alphafold2_code") as span:
            span.set_attribute("sequence_length", len(sequence))

            prompt = f"""
            Given the following amino acid sequence and analysis, generate a Python script to run AlphaFold2 using the NVIDIA NGC API. Use the template provided below, but modify the parameters in the `data` dictionary based on the analysis.

            Sequence: {sequence[:100]}...

            Analysis: {analysis}

            Template:
            ```python
            #!/usr/bin/env python3
            import os
            import requests
            import time
            from pathlib import Path

            # Variables
            key = os.getenv("NVCF_RUN_KEY") or input("Paste the Run Key: ")
            url = os.getenv("URL", "https://health.api.nvidia.com/v1/biology/deepmind/alphafold2-multimer")
            status_url = os.getenv("STATUS_URL", "https://health.api.nvidia.com/v1/status")

            sequences = [
                "{sequence}"
            ]

            output_file = Path("output.json")

            # Request to predict structure from a list of sequences
            headers = {{
                "content-type": "application/json",
                "Authorization": f"Bearer {{key}}",
                "NVCF-POLL-SECONDS": "5",
            }}
            data = {{
                "sequences": sequences,
                "algorithm": "jackhmmer",
                "e_value": 0.0001,
                "iterations": 1,
                "databases": ["uniref90", "small_bfd", "mgnify"],
                "relax_prediction": True,
                # Add other parameters based on the analysis
            }}

            print("Making request...")
            response = requests.post(url, headers=headers, json=data)

            # Check the status code
            if response.status_code == 200:
                output_file.write_text(response.text)
                print(f"Response output to file: {{output_file}}")
            elif response.status_code == 202:
                print("Request accepted...")
                # Extract reqId header
                req_id = response.headers.get("nvcf-reqid")

                # Poll the /status endpoint
                while True:
                    print("Polling for response...")
                    status_response = requests.get(f"{{status_url}}/{{req_id}}", headers=headers)

                    if status_response.status_code != 202:
                        output_file.write_text(status_response.text)
                        print(f"Response output to file: {{output_file}}")
                        break

                    # Wait before polling again
                    time.sleep(5)
            else:
                print(f"Unexpected HTTP status: {{response.status_code}}")
                print(f"Response: {{response.text}}")
            ```

            Generate the Python script, including detailed comments explaining each parameter choice based on the analysis.
            """

            response = openai_client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that generates Python code for running AlphaFold2."},
                    {"role": "user", "content": prompt}
                ]
            )

            code = response.choices[0].message.content
            return code
    except Exception as e:
        logger.error(f"Error generating AlphaFold2 code: {str(e)}")
        raise gr.Error(f"Error generating AlphaFold2 code: {str(e)}")

def alphafold2_app(sequence):
    try:
        with tracing.start_as_current_span("alphafold2_app") as span:
            if not sequence:
                raise gr.Error("No sequence provided. Please enter an amino acid sequence.")

            # Remove any whitespace and validate the sequence
            sequence = ''.join(sequence.split())
            valid_aa = set('ACDEFGHIKLMNPQRSTVWY')
            if not set(sequence).issubset(valid_aa):
                raise gr.Error("Invalid amino acid sequence. Please enter a valid sequence using standard one-letter amino acid codes.")

            span.set_attribute("sequence_length", len(sequence))
            span.set_attribute("sequence_valid", True)

            # Analyze the sequence using GPT-4
            analysis = analyze_sequence_with_gpt4(query_engine, sequence)

            # Generate AlphaFold2 code
            code = generate_alphafold2_code(sequence, analysis)

            return analysis, code
    except gr.Error as e:
        # Re-raise Gradio errors to display them in the UI
        raise
    except Exception as e:
        logger.error(f"Unexpected error in alphafold2_app: {str(e)}")
        logger.error(traceback.format_exc())
        raise gr.Error(f"An unexpected error occurred: {str(e)}")

# Add these GROMACS-specific functions before your main Gradio interface definition
def analyze_pdb_structure(file_path):
    try:
        # Initialize PDB parser
        parser = PDB.PDBParser(QUIET=True)
        structure = parser.get_structure('protein', file_path)

        # Extract basic information
        chains = list(structure.get_chains())
        residues = list(structure[0].get_residues())
        atoms = list(structure.get_atoms())

        # Calculate structure properties
        n_chains = len(chains)
        n_residues = len(residues)
        n_atoms = len(atoms)

        # Try to get secondary structure information
        try:
            dssp_dict = dssp_dict_from_pdb_file(file_path)
            ss_composition = {}
            for residue in dssp_dict:
                ss = dssp_dict[residue][2]
                ss_composition[ss] = ss_composition.get(ss, 0) + 1
        except Exception as e:
            ss_composition = {"Error": "Could not calculate secondary structure"}

        structure_info = {
            "chains": n_chains,
            "residues": n_residues,
            "atoms": n_atoms,
            "secondary_structure": ss_composition
        }

        # Create a text summary
        summary = (
            f"Structure Analysis:\n"
            f"- Number of chains: {n_chains}\n"
            f"- Number of residues: {n_residues}\n"
            f"- Number of atoms: {n_atoms}\n"
            f"- Secondary structure composition: {ss_composition}\n"
        )

        logger.info(f"Successfully analyzed PDB structure: {file_path}")
        return summary
    except Exception as e:
        logger.error(f"Error analyzing PDB structure: {str(e)}")
        raise

def get_nvidia_info(query_engine, structure_info):
    try:
        query = f"Provide information on NVIDIA tools and optimizations relevant to running GROMACS simulations for a protein structure with the following characteristics: {structure_info}"
        response = query_engine.query(query)
        logger.info("Successfully retrieved NVIDIA information")
        return str(response)
    except Exception as e:
        logger.error(f"Error querying NVIDIA information: {str(e)}")
        raise

def process_structure(file_path, query_engine, llm, deepseek_client) -> Tuple[Protocol, str]:
    try:
        # Analyze PDB structure
        structure_info = analyze_pdb_structure(file_path)
        nvidia_info = get_nvidia_info(query_engine, structure_info)

        # Create initial protocol with GPT-4o
        protocol_prompt = (
            f"You are an expert in GROMACS molecular dynamics simulations and NVIDIA scientific computing tools. "
            f"Create a detailed step-by-steo GROMACS simulation protocol for the following protein structure, "
            f"incorporating NVIDIA-specific tools and optimizations where appropriate. "
            f"Consider the structure's characteristics when designing the protocol.\n\n"
            f"Structure information:\n{structure_info}\n\n"
            f"NVIDIA tool information: {nvidia_info}\n\n"
            f"Create a detailed GROMACS protocol for this structure, including system preparation, "
            f"equilibration, and production MD steps. Include specific NVIDIA optimizations."
        )

        # Use the complete() method directly
        protocol_response = llm.complete(protocol_prompt)
        initial_steps = protocol_response.text.split('\n')

        # Evaluate and improve protocol using DeepSeek
        improved_steps = evaluate_protocol_with_deepseek(initial_steps, deepseek_client)
        protocol_text = '\n'.join(improved_steps)

        # Generate NVIDIA benefits description
        benefits_prompt = (
            f"Based on the improved GROMACS protocol and the structure information provided, "
            f"describe in detail the advantages of using NVIDIA technology for this specific "
            f"protein simulation. Focus on performance improvements, specific NVIDIA features "
            f"utilized, and how they enhance the simulation process.\n\n"
            f"Structure information: {structure_info}\n"
            f"Protocol: {protocol_text}\n\n"
            f"NVIDIA tool information: {nvidia_info}\n\n"
            f"Provide a comprehensive explanation of the benefits of NVIDIA technology in this context."
        )

        benefits_response = llm.complete(benefits_prompt)

        logger.info("Successfully generated improved GROMACS protocol and NVIDIA benefits description")
        return Protocol(steps=improved_steps), benefits_response.text
    except Exception as e:
        logger.error(f"Error processing structure: {str(e)}")
        raise

def evaluate_protocol_with_deepseek(protocol_steps: List[str], deepseek_client: OpenAI) -> List[str]:
    try:
        protocol_text = '\n'.join(protocol_steps)
        evaluation_prompt = (
            f"Analyze this GROMACS simulation protocol and create a new detailed GROMACS protocol for this structure, including system preparation, "
            f"include gromacs commands for system preparation, equilibration, and production MD steps. Include specific NVIDIA optimizations."
            f"Current protocol steps:\n"
            f"{protocol_text}\n\n"
            f"focusing on optimal use of modern GROMACS features that can be enhanced with NVIDIA GPUs."
        )

        completion = deepseek_client.chat.completions.create(
            model="deepseek-ai/deepseek-coder-6.7b-instruct",
            messages=[{"role": "user", "content": evaluation_prompt}],
            temperature=0.0,
            top_p=1,
            max_tokens=1024
        )

        improved_steps = completion.choices[0].message.content.split('\n')
        improved_steps = [step.strip() for step in improved_steps if step.strip()]

        logger.info("Successfully evaluated and improved protocol with DeepSeek")
        return improved_steps
    except Exception as e:
        logger.error(f"Error in DeepSeek evaluation: {str(e)}")
        return protocol_steps

# Modify your gromacs_interface function to pass the llm parameter
def gromacs_interface(file):
    try:
        if file is None:
            return "Please upload a PDB file.", "No file uploaded."

        if hasattr(file, 'name'):
            file_path = file.name
        else:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdb") as temp_file:
                temp_file.write(file.read() if hasattr(file, 'read') else file)
                file_path = temp_file.name

        logger.info(f"Processing file: {file_path}")
        protocol, nvidia_benefits = process_structure(file_path, query_engine, llm, deepseek_client)

        if file_path != getattr(file, 'name', None):
            os.unlink(file_path)

        protocol_output = "\n".join(f"{i+1}. {step}" for i, step in enumerate(protocol.steps))
        return protocol_output, nvidia_benefits
    except Exception as e:
        error_msg = f"An error occurred: {str(e)}\n\nPlease check the logs for more details."
        logger.error(error_msg)
        return error_msg, error_msg

def get_diffdock_parameters(protein_content, ligand_smiles):
    try:
        query = f"""
        Analyze the following protein structure and ligand SMILES string and suggest appropriate parameters for running DiffDock:

        Protein structure: {protein_content[:500]}...
        Ligand SMILES: {ligand_smiles}

        Provide the following parameters:
        1. num_poses: Number of poses to generate (typically 20-100)
        2. time_divisions: Number of time divisions for diffusion (typically 20)
        3. steps: Number of diffusion steps (typically 18-25)
        4. save_trajectory: Whether to save the diffusion trajectory
        5. is_staged: Whether to use staged diffusion
        6. batch_size: Batch size for processing

        Explain your reasoning for each parameter.
        """
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert in molecular docking using DiffDock."},
                {"role": "user", "content": query}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Error in GPT-4o analysis: {str(e)}")
        raise gr.Error(f"Error in GPT-4o analysis: {str(e)}")

def generate_diffdock_code(protein_content, ligand_smiles, parameters):
    try:
        # Parse parameters from the analysis
        params = {}
        for line in parameters.split('\n'):
            if ':' in line:
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()

                if key in ['num_poses', 'time_divisions', 'steps']:
                    params[key] = int(value) if value.isdigit() else 20
                elif key in ['save_trajectory', 'is_staged']:
                    params[key] = value.lower() == 'true'

        code = f"""
import requests
import time
import os

def upload_asset(input_data, auth_header):
    assets_url = "https://api.nvcf.nvidia.com/v2/nvcf/assets"
    headers = {{
        "Authorization": auth_header,
        "Content-Type": "application/json",
        "accept": "application/json",
    }}
    s3_headers = {{
        "x-amz-meta-nvcf-asset-description": "diffdock-file",
        "content-type": "text/plain",
    }}
    payload = {{
        "contentType": "text/plain",
        "description": "diffdock-file"
    }}

    response = requests.post(assets_url, headers=headers, json=payload)
    response.raise_for_status()

    asset_url = response.json()["uploadUrl"]
    asset_id = response.json()["assetId"]

    response = requests.put(asset_url, data=input_data, headers=s3_headers)
    response.raise_for_status()

    return asset_id

def run_diffdock(api_key):
    url = "https://health.api.nvidia.com/v1/biology/mit/diffdock"
    header_auth = f"Bearer {{api_key}}"

    # Upload protein structure
    protein_content = \"\"\"
{protein_content}
    \"\"\"
    protein_id = upload_asset(protein_content, header_auth)

    # Upload ligand SMILES
    ligand_content = \"{ligand_smiles}\"
    ligand_id = upload_asset(ligand_content, header_auth)

    headers = {{
        "Content-Type": "application/json",
        "NVCF-INPUT-ASSET-REFERENCES": ",".join([protein_id, ligand_id]),
        "Authorization": header_auth
    }}

    payload = {{
        "ligand": ligand_id,
        "ligand_file_type": "smiles",
        "protein": protein_id,
        "num_poses": {params.get('num_poses', 20)},
        "time_divisions": {params.get('time_divisions', 20)},
        "steps": {params.get('steps', 18)},
        "save_trajectory": {str(params.get('save_trajectory', True)).lower()},
        "is_staged": {str(params.get('is_staged', True)).lower()}
    }}

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()

    return response.json()

if __name__ == "__main__":
    api_key = os.getenv("NVIDIA_API_KEY") or input("Enter your NVIDIA API key: ")
    result = run_diffdock(api_key)
    print(result)
"""
        return code
    except Exception as e:
        logger.error(f"Error generating DiffDock code: {str(e)}")
        raise gr.Error(f"Error generating DiffDock code: {str(e)}")

def execute_diffdock_script(api_key: str, protein_content: str, ligand_smiles: str, parameters: str) -> tuple:
    """
    Execute DiffDock with enhanced error handling and input validation.
    """
    try:
        # Input validation with detailed logging
        if not api_key or not api_key.strip():
            raise ValueError("API key is required")

        if not protein_content or not protein_content.strip():
            raise ValueError("Protein content is empty")

        if not ligand_smiles or not ligand_smiles.strip():
            raise ValueError("Ligand SMILES is empty")

        logger.info(f"Starting DiffDock execution with:"
                   f"\nProtein content length: {len(protein_content)}"
                   f"\nLigand SMILES length: {len(ligand_smiles)}")

        # Define API endpoints
        assets_url = "https://api.nvcf.nvidia.com/v2/nvcf/assets"
        diffdock_url = "https://health.api.nvidia.com/v1/biology/mit/diffdock"

        # Set up headers
        headers = {
            "Authorization": f"Bearer {api_key}",
            "accept": "application/json",
            "content-type": "application/json"
        }

        def upload_asset(content, description="diffdock-file"):
            logger.info(f"Uploading {description} (content length: {len(content)})")

            # Request upload URL
            payload = {
                "contentType": "text/plain",
                "description": description
            }
            upload_response = requests.post(assets_url, headers=headers, json=payload)

            if upload_response.status_code != 200:
                logger.error(f"Asset upload URL request failed: {upload_response.text}")
                raise ValueError(f"Failed to get upload URL: {upload_response.status_code}")

            upload_data = upload_response.json()
            asset_url = upload_data["uploadUrl"]
            asset_id = upload_data["assetId"]

            # Upload content
            s3_headers = {
                "x-amz-meta-nvcf-asset-description": description,
                "content-type": "text/plain",
            }
            put_response = requests.put(asset_url, data=content, headers=s3_headers)

            if put_response.status_code != 200:
                logger.error(f"Asset upload failed: {put_response.text}")
                raise ValueError(f"Failed to upload asset: {put_response.status_code}")

            logger.info(f"Successfully uploaded {description} with asset ID: {asset_id}")
            return asset_id

        # Upload protein and ligand with error checking
        try:
            protein_id = upload_asset(protein_content, "protein-structure")
            ligand_id = upload_asset(ligand_smiles, "ligand-smiles")
        except Exception as upload_error:
            logger.error(f"Asset upload failed: {str(upload_error)}")
            return f"Error uploading assets: {str(upload_error)}", None, False

        # Parse parameters with validation
        parsed_params = {
            "num_poses": 20,
            "time_divisions": 20,
            "steps": 18,
            "save_trajectory": False,
            "is_staged": True
        }

        try:
            for line in parameters.split('\n'):
                if ':' in line and not line.startswith('-'):
                    key, value = line.split(':', 1)
                    key = key.strip()
                    value = value.strip()

                    if key == 'num_poses':
                        value = int(value) if value.isdigit() else 20
                        parsed_params[key] = max(1, min(100, value))
                    elif key == 'time_divisions':
                        value = int(value) if value.isdigit() else 20
                        parsed_params[key] = max(3, min(20, value))
                    elif key == 'steps':
                        value = int(value) if value.isdigit() else 18
                        parsed_params[key] = max(1, min(18, value))
                    elif key in ['save_trajectory', 'is_staged']:
                        parsed_params[key] = value.lower() == 'true'
        except Exception as parse_error:
            logger.error(f"Parameter parsing failed: {str(parse_error)}")
            return f"Error parsing parameters: {str(parse_error)}", None, False

        # Prepare payload
        payload = {
            "ligand": ligand_id,
            "ligand_file_type": "smiles",
            "protein": protein_id,
            "num_poses": parsed_params["num_poses"],
            "time_divisions": parsed_params["time_divisions"],
            "steps": parsed_params["steps"],
            "save_trajectory": parsed_params["save_trajectory"],
            "is_staged": True  # Always true when using asset IDs
        }

        logger.info(f"Making DiffDock API request with configuration:"
                   f"\nnum_poses: {payload['num_poses']}"
                   f"\ntime_divisions: {payload['time_divisions']}"
                   f"\nsteps: {payload['steps']}")

        # Make request with detailed error handling
        response = requests.post(diffdock_url, json=payload, headers=headers)

        # Log the full response for debugging
        logger.info(f"DiffDock API response status: {response.status_code}")
        logger.info(f"DiffDock API response headers: {response.headers}")
        try:
            response_json = response.json()
            logger.info(f"DiffDock API response body: {response_json}")
        except:
            logger.info(f"DiffDock API response text: {response.text}")

        if response.status_code != 200:
            error_msg = f"DiffDock API error (status {response.status_code}): {response.text}"
            logger.error(error_msg)
            return error_msg, None, False

        result = response.json()

        # Process results
        output_files = {}
        if "docked_poses" in result:
            poses_file = "diffdock_poses.sdf"
            with open(poses_file, 'w') as f:
                f.write(result["docked_poses"])
            output_files["poses"] = poses_file

        if parsed_params["save_trajectory"] and "trajectory" in result:
            traj_file = "diffdock_trajectory.pdb"
            with open(traj_file, 'w') as f:
                f.write(result["trajectory"])
            output_files["trajectory"] = traj_file

        status_message = (
            f"DiffDock execution completed!\n"
            f"Configuration:\n"
            f"- Generated poses: {parsed_params['num_poses']}\n"
            f"- Time divisions: {parsed_params['time_divisions']}\n"
            f"- Steps: {parsed_params['steps']}\n\n"
            f"Output files:\n"
            f"- Poses: {output_files.get('poses', 'Not available')}\n"
            f"- Trajectory: {output_files.get('trajectory', 'Not available')}\n"
        )

        logger.info("DiffDock execution completed successfully")
        return status_message, output_files, True

    except Exception as e:
        error_msg = f"Error in DiffDock execution: {str(e)}"
        logger.error(error_msg)
        logger.error(f"Full traceback: {traceback.format_exc()}")
        return error_msg, None, False

def diffdock_app(protein_file, ligand_smiles):
    try:
        if protein_file is None:
            raise gr.Error("Please upload a protein structure file (PDB format).")
        if not ligand_smiles:
            raise gr.Error("Please enter a ligand SMILES string.")

        protein_content = process_pdb_file(protein_file)
        parameters = get_diffdock_parameters(protein_content, ligand_smiles)
        code = generate_diffdock_code(protein_content, ligand_smiles, parameters)

        return parameters, code

    except gr.Error as e:
        raise
    except Exception as e:
        logger.error(f"Unexpected error in diffdock_app: {str(e)}")
        raise gr.Error(f"An unexpected error occurred: {str(e)}")

# Custom CSS for styling (your existing CSS)
custom_css = """
<style>
    body {
        background-color: #1a1a1a;
        color: #ffffff;
    }
    .container {
        max-width: 1200px;
        margin: 0 auto;
    }
    .gr-button {
        background-color: #76b900 !important;
        border: none !important;
    }
    .gr-button:hover {
        background-color: #8ede00 !important;
    }
    .gr-form {
        background-color: #2a2a2a;
        border-radius: 10px;
        padding: 20px;
        margin-bottom: 20px;
    }
    .gr-box {
        background-color: #2a2a2a;
        border-radius: 10px;
        padding: 20px;
        margin-bottom: 20px;
    }
    .gr-paddle {
        background-color: #2a2a2a !important;
    }
    h1, h2, h3 {
        color: #76b900;
    }
</style>
"""

# Initialize tracing and start the application
try:
    # Initialize tracing
    tracing.initialize_tracing()

    # Set up Pinecone index
    vector_store_index = setup_pinecone_index()
    query_engine = vector_store_index.as_query_engine()

    # Initialize audio handler and chat monitor
    audio_handler = AudioResponseHandler(openai_client)
    chat_monitor = ChatSystemMonitor()

    # Create and launch Gradio interface
    with tracing.start_as_current_span("gradio_interface_setup") as span:
        with gr.Blocks(css=custom_css) as iface:
            gr.HTML("""
                <div style="text-align: center; max-width: 800px; margin: 0 auto;">
                    <h1>🧬 Protein Design and Drug Discovery Research Assistant</h1>
                    <p>Powered by AI and Molecular Modeling</p>
                </div>
            """)

            with gr.Tabs() as tabs:
                with gr.Tab("Home"):
                    with gr.Row():
                        with gr.Column():
                            gr.Markdown("""
                            ## Welcome to the Protein Design and Drug Discovery Research Assistant

                            This advanced tool combines cutting-edge AI models, molecular modeling techniques, and NVIDIA's latest technologies to streamline protein design and drug discovery workflows. The interface is organized into eight specialized tabs:

                            Tab 1: NVIDIA Inference Microservices (NIMs) Home

                            Overview of the application and its capabilities
                            Introduction to NVIDIA Inference Microservices (NIMs)
                            Interactive AI chat assistant for general guidance
                            Quick-start guides and tips for getting started
                            System status monitoring and voice interaction options

                            Tab 2: Literature Search

                            Search and analyze scientific papers from arXiv and PubMed
                            AI-powered paper summarization with NVIDIA integration analysis
                            Voice-enabled discussion of research findings
                            Focus on papers relevant to protein design and drug discovery
                            Integration with NVIDIA knowledge base for technical insights

                            Tab 3: PDB Tools

                            Comprehensive Protein Data Bank (PDB) file analysis
                            Interactive 3D structure visualization
                            Abstract lookup for quick protein information
                            Direct PDB file download capabilities
                            AI-assisted structure analysis with voice support

                            Tab 4: RFdiffusion (Protein Design Step 1)

                            Generate novel protein structures using RoseTTAFold Diffusion
                            GPU-accelerated protein backbone generation
                            Customizable parameters for structure generation
                            Integration with NVIDIA's protein design pipeline
                            Real-time visualization of generated structures

                            Tab 5: ProteinMPNN (Protein Design Step 2)

                            Optimize amino acid sequences for generated structures
                            Advanced sequence prediction using neural networks
                            GPU-accelerated sequence optimization
                            Multiple sequence generation with confidence scores
                            Direct integration with RFdiffusion outputs

                            Tab 6: AlphaFold2 (Protein Design Step 3)

                            State-of-the-art protein structure prediction
                            GPU-optimized implementation via NVIDIA NGC
                            Customizable prediction parameters
                            Confidence score analysis
                            Integration with previous design steps

                            Tab 7: MolMIM

                            Molecular generation and optimization using MolMIM
                            NVIDIA-powered molecular design
                            SMILES-based molecule generation
                            Property optimization capabilities
                            Integration with drug discovery workflows

                            Tab 8: DiffDock

                            Advanced molecular docking using diffusion models
                            Blind docking capabilities without pocket information
                            GPU-accelerated pose generation
                            Integrated scoring and ranking
                            Visualization of docking results

                            Tab 9: Molecular Dynamics with GROMACS

                            GPU-optimized molecular dynamics simulations
                            Automated protocol generation
                            NVIDIA technology integration
                            DeepSeek-enhanced optimization
                            Complete workflow support

                            Each tab features an AI assistant with voice interaction capabilities, providing expert guidance and real-time support for your research needs. The system is fully integrated with NVIDIA's GPU acceleration technologies for optimal performance.
                            """)

                        with gr.Column():
                            gr.Markdown("""
                            ## Getting Started with NVIDIA Inference Microservices (NIMs)

                            To get started:
                            1. Create a free account with NVIDIA at https://build.nvidia.com/explore/discover
                            2. Click on your model of choice.
                            3. Under Input select the Python tab, and click `Get API Key`
                            4. Copy and save the generated key as NVIDIA_API_KEY
                            """)

                    gr.Markdown("## Nvidia NIMs Chat Assistant")
                    gr.Markdown("""Need help getting started? Try asking some of these questions to the AI assistant below:
                                - What are Nvidia inference microservices (NIMS)?
                                - What are the benefits of Nvidia inference microservices (NIMS)?
                                - How can Nvidia inference microservices (NIMS) be used in Protein Design?
                                - How can Nvidia inference microservices (NIMS) be used in Drug Discovery?""")

                    # Updated chat interface with monitoring
                    with gr.Row():
                        with gr.Column(scale=4):
                            nims_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False
                            )
                            with gr.Row():
                                nims_msg = gr.Textbox(
                                    placeholder="Ask about Nvidia NIMs or protein design...",
                                    show_label=False,
                                    container=False,
                                    scale=8
                                )
                                nims_clear = gr.ClearButton(
                                    components=[nims_msg, nims_chatbot],
                                    scale=1
                                )

                        with gr.Column(scale=1):
                            system_status = gr.JSON(
                                label="System Status",
                                value=lambda: chat_monitor.get_status()
                            )
                            voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                container=True
                            )
                            audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath"
                            )

                    # Chat handler with monitoring
                    def chat_handler(message, history, state):
                        try:
                            with tracing.start_as_current_span("chat_interaction") as chat_span:
                                chat_span.set_attribute("message_length", len(message))
                                result = chat_with_gpt4o(message, history, state)
                                chat_monitor.record_success()
                                return result
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Chat handler error: {str(e)}")
                            raise gr.Error(f"Chat system error: {str(e)}")

                    # Voice update handler with monitoring
                    def update_voice_handler(voice):
                        try:
                            audio_handler.set_voice(voice)
                            chat_monitor.record_success()
                            return None
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Voice update error: {str(e)}")
                            raise gr.Error(f"Voice update error: {str(e)}")

                    # Connect components with error handling
                    nims_msg.submit(
                        chat_handler,
                        inputs=[nims_msg, nims_chatbot, gr.State([])],
                        outputs=[nims_msg, nims_chatbot, gr.State([]), audio_output]
                    ).then(
                        lambda: chat_monitor.get_status(),
                        outputs=[system_status]
                    )

                    voice_selector.change(
                        update_voice_handler,
                        inputs=[voice_selector],
                        outputs=[audio_output]
                    ).then(
                        lambda: chat_monitor.get_status(),
                        outputs=[system_status]
                    )

                with gr.TabItem("Literature Search"):
                    gr.Markdown("""
                    ## Scientific Literature Search and Analysis

                    Search and analyze scientific papers from arXiv and PubMed. The AI assistant can help you understand the papers
                    and relate them to NVIDIA technologies and implementations.

                    Features:
                    - Search papers across multiple scientific databases
                    - Get AI-generated summaries of papers
                    - Voice-enabled chat assistance
                    - Integration with NVIDIA knowledge base

                    Try searching some of these that use the generative models and Nvidia technologies you can learn how to use in this application:
                    - Improving Small Molecule Generation using Mutual Information Machine (in Arxiv)
                    - PLINDER: The protein-ligand interactions dataset and evaluation resource (in Pubmed)


                    """)

                    # Paper Search Section
                    with gr.Row():
                        with gr.Column():
                            search_query = gr.Textbox(
                                label="Search Query",
                                placeholder="Enter keywords for paper search...",
                                lines=2
                            )
                            search_source = gr.Radio(
                                choices=["arxiv", "pubmed"],
                                label="Source Database",
                                value="arxiv",
                                interactive=True
                            )
                            search_button = gr.Button("🔍 Search Papers", variant="primary")

                        with gr.Column():
                            paper_summaries = gr.Textbox(
                                label="Paper Summaries and Analysis",
                                lines=10,
                                interactive=False
                            )
                            summary_audio = gr.Audio(
                                label="Summary Audio",
                                type="filepath",
                                visible=True
                            )

                    # Chat Assistant Section
                    gr.Markdown("## Literature Chat Assistant")
                    gr.Markdown("""
                    Discuss the papers with our AI assistant. It can help you:
                    - Understand complex concepts
                    - Relate findings to NVIDIA technologies
                    - Explain technical details
                    - Compare different research approaches
                    """)

                    with gr.Row():
                        # Chat Interface
                        with gr.Column(scale=4):
                            literature_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False,
                                container=False
                            )
                            with gr.Row():
                                literature_msg = gr.Textbox(
                                    placeholder="Ask about the papers or related NVIDIA technologies...",
                                    show_label=False,
                                    container=False,
                                    scale=9
                                )
                                literature_clear = gr.ClearButton(
                                    components=[literature_msg, literature_chatbot],
                                    scale=1,
                                    variant="secondary"
                                )

                        # Voice and Audio Controls
                        with gr.Column(scale=1):
                            voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                info="Choose the AI assistant's voice",
                                container=True
                            )
                            audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath",
                                show_label=True,
                                container=True
                            )

                        def literature_chat(message, chat_history, history_state, paper_summaries):
                            """
                            Enhanced literature chat assistant with proper flow control and error handling
                            """
                            try:
                                if history_state is None:
                                    history_state = []

                                logger.info("Starting literature chat interaction")

                                # Phase 1: Query Preparation
                                try:
                                    logger.info("Preparing query and context")
                                    # Create a synchronous query to prevent stalling
                                    query_result = query_engine.query(
                                        message,
                                        response_mode="compact",  # Use compact mode for faster processing
                                        timeout=30  # Add timeout to prevent hanging
                                    )
                                    relevant_context = str(query_result.response)
                                    logger.info("Context retrieved successfully")
                                except Exception as e:
                                    logger.error(f"Error in query phase: {str(e)}")
                                    relevant_context = "Unable to retrieve context. Proceeding with basic response."

                                # Phase 2: Chat Completion
                                try:
                                    logger.info("Initiating chat completion")
                                    system_message = {
                                        "role": "system",
                                        "content": f"""You are an AI assistant specializing in scientific literature analysis and NVIDIA technologies.
                                        Use this information to inform your response:

                                        Paper Summaries: {paper_summaries}
                                        Relevant Context: {relevant_context}

                                        Focus on:
                                        1. Explaining scientific concepts clearly
                                        2. Relating findings to NVIDIA technologies
                                        3. Providing practical insights for implementation
                                        4. Identifying connections between research papers

                                        If you don't have enough information to answer a specific question,
                                        please say so and offer to help with other aspects of the literature or NVIDIA's capabilities."""
                                    }

                                    messages = [system_message] + history_state + [{"role": "user", "content": message}]

                                    response = openai_client.chat.completions.create(
                                        model="gpt-4o",
                                        messages=messages,
                                        max_tokens=16384,
                                        temperature=0,
                                        stream=False,
                                        timeout=60  # Add timeout for chat completion
                                    )

                                    response_text = response.choices[0].message.content
                                    logger.info("Chat completion successful")
                                except Exception as e:
                                    logger.error(f"Error in chat completion: {str(e)}")
                                    return message, chat_history, history_state, None

                                # Phase 3: Audio Generation
                                try:
                                    logger.info("Starting audio generation")
                                    audio_handler = AudioResponseHandler(openai_client)
                                    audio_file_path = audio_handler.text_to_speech(response_text)

                                    if not audio_file_path:
                                        logger.warning("Audio generation returned no file path")
                                        raise ValueError("Audio generation failed")

                                    logger.info(f"Audio generated successfully: {audio_file_path}")
                                except Exception as e:
                                    logger.error(f"Error in audio generation: {str(e)}")
                                    audio_file_path = None
                                    response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

                                # Update conversation state
                                chat_history = chat_history + [(message, response_text)]
                                history_state.append({"role": "assistant", "content": response_text})

                                logger.info("Literature chat interaction completed successfully")
                                return message, chat_history, history_state, audio_file_path

                            except Exception as e:
                                logger.error(f"Critical error in literature chat system: {str(e)}")
                                error_message = "I apologize, but I encountered an unexpected error. Please try again."
                                chat_history = chat_history + [(message, error_message)]
                                return message, chat_history, history_state, None

                        # Update the chat handler with monitoring
                        def literature_chat_handler(message, history, state, paper_summaries):
                            try:
                                with tracing.start_as_current_span("literature_chat_interaction") as chat_span:
                                    chat_span.set_attribute("message_length", len(message))
                                    result = literature_chat(message, history, state, paper_summaries)
                                    chat_monitor.record_success()
                                    return result
                            except Exception as e:
                                chat_monitor.record_failure(e)
                                logger.error(f"Literature chat handler error: {str(e)}")
                                raise gr.Error(f"Chat system error: {str(e)}")

                    # Voice update handler with monitoring
                    def update_literature_voice(voice):
                        try:
                            audio_handler = AudioResponseHandler(openai_client)
                            audio_handler.set_voice(voice)
                            chat_monitor.record_success()
                            return None
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Voice update error: {str(e)}")
                            raise gr.Error(f"Voice update error: {str(e)}")

                    # Connect the chat interface
                    literature_msg.submit(
                        literature_chat,
                        inputs=[
                            literature_msg,
                            literature_chatbot,
                            gr.State([]),
                            paper_summaries
                        ],
                        outputs=[
                            literature_msg,
                            literature_chatbot,
                            gr.State([]),
                            audio_output
                        ]
                    )

                    # Connect the components with proper error handling
                    search_button.click(
                        fn=search_and_summarize_papers,
                        inputs=[search_query, search_source],
                        outputs=[paper_summaries, summary_audio],
                        api_name="search_papers"
                    )

                    # Connect voice selection
                    voice_selector.change(
                        update_voice_handler,
                        inputs=[voice_selector],
                        outputs=[audio_output]
                    )

                    gr.Markdown("""
                    ---
                    ### Tips for Effective Use
                    - Use specific keywords in your search
                    - Include relevant technical terms
                    - Ask follow-up questions for clarification
                    - Use the voice feature for hands-free interaction
                    """)

                with gr.TabItem("Protein Databank (PDB) Tools"):
                    gr.Markdown("""
                    ## PDB Tools: Structure Analysis and Visualization

                    ### Overview
                    The PDB Tools module provides comprehensive tools for working with Protein Data Bank (PDB) files, including:
                    - Abstract lookup for quick protein information
                    - PDB file download capabilities
                    - Interactive 3D structure visualization
                    - AI-powered structure analysis and assistance

                    ### Features
                    - **Abstract Lookup**: Quickly retrieve scientific abstracts for any PDB ID
                    - **File Download**: Direct download of PDB files for local analysis
                    - **3D Visualization**: Interactive molecular visualization with customizable representations
                    - **Voice-Enabled AI Assistant**: Get expert help understanding protein structures
                    - **NVIDIA Integration**: Access specialized analysis tools and GPU-accelerated features

                    ### How to Use
                    1. **For Abstract Lookup**:
                      - Enter a valid PDB ID (e.g., "1ak4")
                      - Click "Lookup PDB Abstract"

                    2. **For File Download**:
                      - Enter a PDB ID
                      - Click "Download PDB File"

                    3. **For Structure Visualization**:
                      - Upload a PDB file
                      - Use the 3D viewer controls to explore the structure

                    4. **For AI Assistance**:
                      - Ask questions about your structure or PDB tools
                      - Use voice interaction for hands-free operation
                    """)

                    # Chat interface with voice components
                    with gr.Row():
                        with gr.Column(scale=4):
                            pdb_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False
                            )
                            pdb_msg = gr.Textbox(
                                placeholder="Ask about PDB structures or analysis tools...",
                                show_label=False,
                                container=False
                            )
                            pdb_clear = gr.ClearButton([pdb_msg, pdb_chatbot])

                        with gr.Column(scale=1):
                            pdb_voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                container=True
                            )
                            pdb_audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath"
                            )

                    gr.Markdown("## PDB Abstract Lookup")
                    with gr.Row():
                        with gr.Column():
                            pdb_id_input = gr.Textbox(label="PDB ID", placeholder="Enter PDB ID...")
                            pdb_lookup_button = gr.Button("🔬 Lookup PDB Abstract", variant="primary")
                        with gr.Column():
                            pdb_abstract = gr.Textbox(label="PDB Abstract", lines=5)

                    gr.Markdown("## Download PDB File")
                    with gr.Row():
                        with gr.Column():
                            pdb_download_id = gr.Textbox(label="PDB ID", placeholder="Enter PDB ID...")
                            pdb_download_button = gr.Button("📥 Download PDB File", variant="primary")
                        with gr.Column():
                            pdb_download_result = gr.Textbox(label="Download Result")

                    gr.Markdown("## Visualize PDB Files")
                    with gr.Row():
                        with gr.Column():
                            pdb_file_input = gr.File(label="Upload PDB File")
                        with gr.Column():
                            reps = [
                                {
                                    "model": 0,
                                    "chain": "",
                                    "resname": "",
                                    "style": "cartoon",
                                    "color": "spectrum",
                                    "residue_range": "",
                                    "around": 0,
                                    "byres": False,
                                    "visible": True
                                },
                                {
                                    "model": 0,
                                    "chain": "",
                                    "resname": "",
                                    "style": "stick",
                                    "color": "element",
                                    "residue_range": "",
                                    "around": 0,
                                    "byres": False,
                                    "visible": False
                                }
                            ]
                            molecule_3d = Molecule3D(label="PDB Visualization", reps=reps)

                    def pdb_chat_with_voice(message, chat_history, history_state, abstract=None, pdb_file=None):
                        """
                        Enhanced PDB chat assistant with proper flow control and error handling
                        """
                        try:
                            if history_state is None:
                                history_state = []

                            logger.info("Starting PDB chat interaction")

                            # Phase 1: Query Preparation
                            try:
                                logger.info("Preparing query and context")
                                # Create a synchronous query to prevent stalling
                                query_result = query_engine.query(
                                    message,
                                    response_mode="compact",  # Use compact mode for faster processing
                                    timeout=30  # Add timeout to prevent hanging
                                )
                                relevant_info = str(query_result.response)
                                logger.info("Context retrieved successfully")
                            except Exception as e:
                                logger.error(f"Error in query phase: {str(e)}")
                                relevant_info = "Unable to retrieve context. Proceeding with basic response."

                            # Phase 2: Create Context
                            context = (
                                f"PDB Abstract: {abstract if abstract else 'No abstract loaded'}\n"
                                f"PDB File: {'Uploaded' if pdb_file else 'No file uploaded'}\n"
                                f"Relevant Information: {relevant_info}"
                            )

                            # Phase 3: Chat Completion
                            try:
                                logger.info("Initiating chat completion")
                                system_message = {
                                    "role": "system",
                                    "content": f"""You are an AI assistant specializing in protein structure analysis and PDB tools.
                                    Use the following information to help answer the user's question:

                                    {context}

                                    Focus on:
                                    1. Analyzing protein structures and features
                                    2. Explaining PDB file formats and tools
                                    3. Providing practical insights for structure analysis
                                    4. Relating findings to protein design and function

                                    If you don't have enough information to answer a specific question,
                                    please say so and offer to help with other aspects of protein structure analysis."""
                                }

                                messages = [system_message] + history_state + [{"role": "user", "content": message}]

                                response = openai_client.chat.completions.create(
                                    model="gpt-4o",
                                    messages=messages,
                                    max_tokens=16384,
                                    temperature=0,
                                    stream=False,
                                    timeout=60  # Add timeout for chat completion
                                )

                                response_text = response.choices[0].message.content
                                logger.info("Chat completion successful")
                            except Exception as e:
                                logger.error(f"Error in chat completion: {str(e)}")
                                return message, chat_history, history_state, None

                            # Phase 4: Audio Generation
                            try:
                                logger.info("Starting audio generation")
                                audio_handler = AudioResponseHandler(openai_client)
                                audio_file_path = audio_handler.text_to_speech(response_text)

                                if not audio_file_path:
                                    logger.warning("Audio generation returned no file path")
                                    raise ValueError("Audio generation failed")

                                logger.info(f"Audio generated successfully: {audio_file_path}")
                            except Exception as e:
                                logger.error(f"Error in audio generation: {str(e)}")
                                audio_file_path = None
                                response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

                            # Update conversation state
                            chat_history = chat_history + [(message, response_text)]
                            history_state.append({"role": "assistant", "content": response_text})

                            logger.info("PDB chat interaction completed successfully")
                            return message, chat_history, history_state, audio_file_path

                        except Exception as e:
                            logger.error(f"Critical error in PDB chat system: {str(e)}")
                            error_message = "I apologize, but I encountered an unexpected error. Please try again."
                            chat_history = chat_history + [(message, error_message)]
                            return message, chat_history, history_state, None

                    # Update the chat handler with monitoring
                    def pdb_chat_handler(message, chat_history, state, abstract, pdb_file):
                        try:
                            with tracing.start_as_current_span("pdb_chat_interaction") as chat_span:
                                chat_span.set_attribute("message_length", len(message))
                                result = pdb_chat_with_voice(message, chat_history, state, abstract, pdb_file)
                                chat_monitor.record_success()
                                return result
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"PDB chat handler error: {str(e)}")
                            raise gr.Error(f"Chat system error: {str(e)}")

                    # Voice selection handler with monitoring
                    def update_pdb_voice_handler(voice):
                        try:
                            audio_handler = AudioResponseHandler(openai_client)
                            audio_handler.set_voice(voice)
                            chat_monitor.record_success()
                            return None
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Voice update error: {str(e)}")
                            raise gr.Error(f"Voice update error: {str(e)}")

                    # Connect the chat function
                    pdb_msg.submit(
                        pdb_chat_handler,
                        inputs=[
                            pdb_msg,
                            pdb_chatbot,
                            gr.State([]),
                            pdb_abstract,
                            pdb_file_input
                        ],
                        outputs=[
                            pdb_msg,
                            pdb_chatbot,
                            gr.State([]),
                            pdb_audio_output
                        ]
                    )

                    # Connect voice selection
                    pdb_voice_selector.change(
                        update_pdb_voice_handler,
                        inputs=[pdb_voice_selector],
                        outputs=[pdb_audio_output]
                    )

                    # Connect existing PDB tools
                    pdb_lookup_button.click(fetch_pdb_abstract, inputs=[pdb_id_input], outputs=pdb_abstract)
                    pdb_download_button.click(download_pdb, inputs=[pdb_download_id], outputs=pdb_download_result)
                    pdb_file_input.change(visualize_pdb, inputs=[pdb_file_input], outputs=[molecule_3d])

                with gr.TabItem("RFdiffusion- Protein Design Step 1"):
                    gr.Markdown("""
                    ## RFdiffusion: Generate New Protein Structures https://build.nvidia.com/ipd/rfdiffusion

                    ### Model Overview
                    RFdiffusion (RoseTTAFold Diffusion) is a generative model that creates novel protein structures for scaffolding and binder design tasks. It generates new protein backbones and designs proteins tailored to bind to target molecules.

                    ### Key Parameters
                    - **input_pdb**: Input PDB file containing protein chains and amino acids for binder target and motifs.
                    - **contigs**: Defines the protein to be generated. Example: 'A10-100/0 50-150' keeps amino acids 10-100 in Chain A, then constructs a new chain of 50-150 amino acids.
                    - **hotspot_res**: Specifies which region the new protein must contact with the original protein. Format: 'A50,A51,A52' for Chain A, positions 50, 51, 52.
                    - **diffusion_steps**: Number of denoising steps (min: 15, default: 50). More steps can improve quality but increase runtime.
                    - **random_seed**: Optional. Sets a fixed seed for reproducible results.

                    ### How to Use
                    1. Upload your PDB file. Because this workflow is for demonstration purposes, it works best for simple, single-chain PDB files. Try these examples to see how RFdiffusion works:
                       - 1AKI Lysozyme https://www.rcsb.org/structure/1aki
                       - 1GTS Glutaminyl-tRNA synthetase https://www.rcsb.org/structure/1GTS
                    2. Click 'Generate RFdiffusion Script' to create a custom python script to run RFdiffusion via the NIM API. An audio file will also be automatically generated to explain the RFdiffusion parameters selected for your protein design run.
                    3. Follow this link to the NIM API catalog to generate a RFdiffusion API key under the Python tab: https://build.nvidia.com/ipd/rfdiffusion?snippet_tab=Python
                    4. Enter you API key in the 'Execute RFdiffusion' interface and click 'Run RFdiffusion' to make the API call. A new protein structure will be generated and appear in the molecular visualization window below.
                    5. A PDB file for your new structure will be automatically downloaded to your file system. Use this PDB file in the ProteinMPNN tab to create new amino acid sequences to fit your new protein structure.
                    """)

                    # Update the chat interface to include voice components
                    with gr.Row():
                        with gr.Column(scale=4):
                            rf_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False
                            )
                            rf_msg = gr.Textbox(
                                placeholder="Ask about RFdiffusion or the uploaded PDB file...",
                                show_label=False,
                                container=False
                            )
                            rf_clear = gr.ClearButton([rf_msg, rf_chatbot])

                        with gr.Column(scale=1):
                            rf_voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                container=True
                            )
                            rf_audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath"
                            )

                    def rf_chat_with_voice(message, chat_history, history_state, parameters, code):
                        """
                        Enhanced RFdiffusion chat assistant with proper flow control and error handling
                        """
                        try:
                            if history_state is None:
                                history_state = []

                            logger.info("Starting RFdiffusion chat interaction")

                            # Phase 1: Query Preparation
                            try:
                                logger.info("Preparing query and context")
                                query_result = query_engine.query(
                                    message,
                                    response_mode="compact",
                                    timeout=30
                                )
                                relevant_info = str(query_result.response)
                                logger.info("Context retrieved successfully")
                            except Exception as e:
                                logger.error(f"Error in query phase: {str(e)}")
                                relevant_info = "Unable to retrieve context. Proceeding with basic response."

                            # Phase 2: Chat Completion
                            try:
                                logger.info("Initiating chat completion")
                                system_message = {
                                    "role": "system",
                                    "content": f"""You are an AI assistant specializing in RFdiffusion and protein structure generation.
                                    Use this relevant information to inform your response: {relevant_info}

                                    Current Context:
                                    Parameters: {parameters}
                                    Code: {code}

                                    Help the user understand:
                                    1. RFdiffusion capabilities and limitations
                                    2. Parameter optimization and settings
                                    3. Integration with NVIDIA GPU acceleration
                                    4. Best practices for protein structure generation

                                    If you don't have enough information to answer a specific question,
                                    please say so and offer to help with other aspects of RFdiffusion or protein design."""
                                }

                                messages = [system_message] + history_state + [{"role": "user", "content": message}]

                                response = openai_client.chat.completions.create(
                                    model="gpt-4o",
                                    messages=messages,
                                    max_tokens=16384,
                                    temperature=0,
                                    stream=False,
                                    timeout=60
                                )

                                response_text = response.choices[0].message.content
                                logger.info("Chat completion successful")
                            except Exception as e:
                                logger.error(f"Error in chat completion: {str(e)}")
                                return message, chat_history, history_state, None

                            # Phase 3: Audio Generation
                            try:
                                logger.info("Starting audio generation")
                                audio_handler = AudioResponseHandler(openai_client)
                                audio_file_path = audio_handler.text_to_speech(response_text)

                                if not audio_file_path:
                                    logger.warning("Audio generation returned no file path")
                                    raise ValueError("Audio generation failed")

                                logger.info(f"Audio generated successfully: {audio_file_path}")
                            except Exception as e:
                                logger.error(f"Error in audio generation: {str(e)}")
                                audio_file_path = None
                                response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

                            # Update conversation state
                            chat_history = chat_history + [(message, response_text)]
                            history_state.append({"role": "assistant", "content": response_text})

                            logger.info("RFdiffusion chat interaction completed successfully")
                            return message, chat_history, history_state, audio_file_path

                        except Exception as e:
                            logger.error(f"Critical error in RFdiffusion chat system: {str(e)}")
                            error_message = "I apologize, but I encountered an unexpected error. Please try again."
                            chat_history = chat_history + [(message, error_message)]
                            return message, chat_history, history_state, None

                    # Handler function with monitoring
                    def rf_chat_handler(message, chat_history, state, parameters, code):
                        try:
                            with tracing.start_as_current_span("rf_chat_interaction") as chat_span:
                                chat_span.set_attribute("message_length", len(message))
                                result = rf_chat_with_voice(message, chat_history, state, parameters, code)
                                chat_monitor.record_success()
                                return result
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"RFdiffusion chat handler error: {str(e)}")
                            raise gr.Error(f"Chat system error: {str(e)}")

                    # Voice update handler with monitoring
                    def update_rf_voice_handler(voice):
                        try:
                            audio_handler = AudioResponseHandler(openai_client)
                            audio_handler.set_voice(voice)
                            chat_monitor.record_success()
                            return None
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Voice update error: {str(e)}")
                            raise gr.Error(f"Voice update error: {str(e)}")

                    # Add voice components for RFdiffusion parameter analysis
                    with gr.Row():
                        with gr.Column():
                            rf_file_input = gr.File(label="Upload PDB File")
                            rf_generate_button = gr.Button("🧬 Generate RFdiffusion Script", variant="primary")
                        with gr.Column():
                            rf_parameters_output = gr.Textbox(label="RFdiffusion Parameters (AI Analysis)", lines=10)
                            rf_code_output = gr.Code(language="python", label="RFdiffusion Python Code")
                            rf_explanation_output = gr.Textbox(
                                label="Parameter Explanation",
                                lines=5,
                                interactive=False
                            )
                            rf_audio_output = gr.Audio(
                                label="Listen to Parameter Analysis",
                                type="filepath"
                            )

                    gr.Markdown("## Execute RFdiffusion")
                    with gr.Row():
                        with gr.Column():
                            api_key_input = gr.Textbox(
                                label="NVIDIA API Key",
                                placeholder="Enter your NVIDIA API key...",
                                type="password"
                            )
                            execute_button = gr.Button("🚀 Run RFdiffusion", variant="primary")
                        with gr.Column():
                            execution_output = gr.Textbox(
                                label="Execution Results",
                                lines=10,
                                interactive=False
                            )

                    # Structure visualization section
                    gr.Markdown("## Generated Structure")
                    with gr.Row():
                        with gr.Column():
                            output_file = gr.File(
                                label="Download Generated Structure",
                                visible=True,
                                interactive=False
                            )
                        with gr.Column():
                            molecule_viewer = Molecule3D(
                                label="Structure Visualization",
                                reps=[{
                                    "model": 0,
                                    "chain": "",
                                    "style": "cartoon",
                                    "color": "spectrum",
                                }]
                            )

                    def run_rfdiffusion(api_key, file, parameters):
                        """
                        Executes RFdiffusion with proper parameter handling

                        Args:
                            api_key (str): NVIDIA API key
                            file: Uploaded PDB file
                            parameters (str): AI-analyzed parameters from get_rfdiffusion_parameters
                        """
                        try:
                            if file is None:
                                return "Please upload a PDB file first.", None, None

                            if not parameters:
                                return "Please generate parameters first by clicking 'Generate RFdiffusion Script'.", None, None

                            pdb_content = process_pdb_file(file)
                            if not pdb_content:
                                return "Error: Could not read PDB file.", None, None

                            # Execute RFdiffusion with parameters
                            status, output_pdb, success = execute_rfdiffusion_script(api_key, pdb_content, parameters)

                            if not success:
                                return status, None, None

                            # Create temporary file for visualization and download
                            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.pdb', mode='w')
                            temp_file.write(output_pdb)
                            temp_file.close()

                            return status, temp_file.name, temp_file.name

                        except Exception as e:
                            logger.error(f"Error in RFdiffusion execution: {str(e)}")
                            return f"Error: {str(e)}", None, None

                    # Update the message submission to use the new handler
                    rf_msg.submit(
                        rf_chat_handler,
                        inputs=[
                            rf_msg,
                            rf_chatbot,
                            gr.State([]),
                            rf_parameters_output,
                            rf_code_output
                        ],
                        outputs=[
                            rf_msg,
                            rf_chatbot,
                            gr.State([]),
                            rf_audio_output
                        ]
                    )

                    # Update voice selection to use the new handler
                    rf_voice_selector.change(
                        update_rf_voice_handler,
                        inputs=[rf_voice_selector],
                        outputs=[rf_audio_output]
                    )

                    # Connect the execute button with the updated run_rfdiffusion function
                    execute_button.click(
                        fn=run_rfdiffusion,
                        inputs=[
                            api_key_input,
                            rf_file_input,
                            rf_parameters_output  # Add parameters from the analysis
                        ],
                        outputs=[
                            execution_output,
                            output_file,
                            molecule_viewer
                        ]
                    )

                    # Update the generation button click event
                    rf_generate_button.click(
                        rfdiffusion_app,
                        inputs=[rf_file_input],
                        outputs=[
                            rf_parameters_output,
                            rf_code_output,
                            rf_explanation_output,
                            rf_audio_output
                        ]
                    )

                with gr.TabItem("ProteinMPNN- Protein Design Step 2"):
                    gr.Markdown("""
                    ## ProteinMPNN: Optimize Amino Acid Sequences https://build.nvidia.com/ipd/proteinmpnn

                    ### Model Overview
                    ProteinMPNN (Protein Message Passing Neural Network) is a deep learning model that predicts amino acid sequences for given protein backbones. It uses evolutionary, functional, and structural information to generate sequences likely to fold into desired 3D structures.

                    ### Key Parameters
                    - **input_pdb**: Input protein structure in PDB format.
                    - **input_pdb_chains**: Specify chains to design (default: all chains).
                    - **ca_only**: Use CA-only model, focusing on alpha carbon atoms (default: false).
                    - **use_soluble_model**: Choose between soluble and non-soluble models (default: false).
                    - **num_seq_per_target**: Number of sequences to generate per target (default: 1).
                    - **sampling_temp**: Controls design diversity (range: 0.1 to 0.3, higher = more diverse).
                    - **random_seed**: Set for reproducibility, omit for diverse results.

                    ### Advanced Options
                    - **pssm_jsonl**: Incorporate evolutionary information to guide mutations.
                    - **fixed_positions_jsonl**: Specify residues to remain unchanged.
                    - **omit_AAs**: Exclude specific amino acids from the design.
                    - **bias_AA_jsonl**: Fine-tune amino acid composition with a bias dictionary.
                    - **tied_positions_jsonl**: Ensure specific residues are identical across positions/chains.

                    ### How to Use
                    1. Upload your PDB file created with RFdiffusion.
                    2. Follow this link to the NIM API catalog to generate a ProteinMPNN API key under the Python tab: https://build.nvidia.com/ipd/proteinmpnn?snippet_tab=Python
                    3. Input your API key and click 'Generate and Run ProteinMPNN Script' to create a custom script for your sequence optimization task and make the API call. A new amino acid sequence in FASTA format will be generated and appear in the execution results window.
                    4. Copy and paste the amino acid sequence in to the AlphaFold2 interface or the AlphaFold3 server to create your new protein structure.
                    """)

                    # Add voice components for ProteinMPNN tab
                    with gr.Row():
                        with gr.Column(scale=4):
                            pmpnn_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False
                            )
                            pmpnn_msg = gr.Textbox(
                                placeholder="Ask about ProteinMPNN or your protein design...",
                                show_label=False,
                                container=False
                            )
                            pmpnn_clear = gr.ClearButton([pmpnn_msg, pmpnn_chatbot])

                        with gr.Column(scale=1):
                            pmpnn_voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                container=True
                            )
                            pmpnn_audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath"
                            )

                    def proteinmpnn_chat(message, chat_history, history_state, parameters, code):
                        """
                        Enhanced ProteinMPNN chat assistant with proper flow control and error handling
                        """
                        try:
                            if history_state is None:
                                history_state = []

                            logger.info("Starting ProteinMPNN chat interaction")

                            # Phase 1: Query Preparation
                            try:
                                logger.info("Preparing query and context")
                                query_result = query_engine.query(
                                    message,
                                    response_mode="compact",
                                    timeout=30
                                )
                                relevant_info = str(query_result.response)
                                logger.info("Context retrieved successfully")
                            except Exception as e:
                                logger.error(f"Error in query phase: {str(e)}")
                                relevant_info = "Unable to retrieve context. Proceeding with basic response."

                            # Phase 2: Chat Completion
                            try:
                                logger.info("Initiating chat completion")
                                system_message = {
                                    "role": "system",
                                    "content": f"""You are an AI assistant specializing in ProteinMPNN and protein sequence design.
                                    Use this relevant information to inform your response:

                                    Relevant Information: {relevant_info}
                                    Parameters: {parameters}
                                    Code: {code}

                                    Focus on:
                                    1. Explaining ProteinMPNN capabilities and limitations
                                    2. Guiding parameter optimization
                                    3. Interpreting sequence design results
                                    4. Best practices for protein sequence design
                                    5. NVIDIA GPU acceleration features

                                    If you don't have enough information to answer a specific question,
                                    please say so and offer to help with other aspects of ProteinMPNN or protein sequence design."""
                                }

                                messages = [system_message] + history_state + [{"role": "user", "content": message}]

                                response = openai_client.chat.completions.create(
                                    model="gpt-4o",
                                    messages=messages,
                                    max_tokens=16384,
                                    temperature=0,
                                    stream=False,
                                    timeout=60
                                )

                                response_text = response.choices[0].message.content
                                logger.info("Chat completion successful")
                            except Exception as e:
                                logger.error(f"Error in chat completion: {str(e)}")
                                return message, chat_history, history_state, None

                            # Phase 3: Audio Generation
                            try:
                                logger.info("Starting audio generation")
                                audio_handler = AudioResponseHandler(openai_client)
                                audio_file_path = audio_handler.text_to_speech(response_text)

                                if not audio_file_path:
                                    logger.warning("Audio generation returned no file path")
                                    raise ValueError("Audio generation failed")

                                logger.info(f"Audio generated successfully: {audio_file_path}")
                            except Exception as e:
                                logger.error(f"Error in audio generation: {str(e)}")
                                audio_file_path = None
                                response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

                            # Update conversation state
                            chat_history = chat_history + [(message, response_text)]
                            history_state.append({"role": "assistant", "content": response_text})

                            logger.info("ProteinMPNN chat interaction completed successfully")
                            return message, chat_history, history_state, audio_file_path

                        except Exception as e:
                            logger.error(f"Critical error in ProteinMPNN chat system: {str(e)}")
                            error_message = "I apologize, but I encountered an unexpected error. Please try again."
                            chat_history = chat_history + [(message, error_message)]
                            return message, chat_history, history_state, None

                    # Add these handler functions right after the proteinmpnn_chat function

                    def proteinmpnn_chat_handler(message, chat_history, state, parameters, code):
                        """Handler function with monitoring"""
                        try:
                            with tracing.start_as_current_span("proteinmpnn_chat_interaction") as chat_span:
                                chat_span.set_attribute("message_length", len(message))
                                result = proteinmpnn_chat(message, chat_history, state, parameters, code)
                                chat_monitor.record_success()
                                return result
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"ProteinMPNN chat handler error: {str(e)}")
                            raise gr.Error(f"Chat system error: {str(e)}")

                    def update_proteinmpnn_voice_handler(voice):
                        """Voice update handler with monitoring"""
                        try:
                            audio_handler = AudioResponseHandler(openai_client)
                            audio_handler.set_voice(voice)
                            chat_monitor.record_success()
                            return None
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Voice update error: {str(e)}")
                            raise gr.Error(f"Voice update error: {str(e)}")

                    gr.Markdown("## Generate and Execute ProteinMPNN")
                    with gr.Row():
                        with gr.Column():
                            pmpnn_file_input = gr.File(label="Upload PDB File")
                            pmpnn_api_key = gr.Textbox(
                                label="NVIDIA API Key (Optional)",
                                placeholder="Enter your NVIDIA API key to run ProteinMPNN...",
                                type="password"
                            )
                            pmpnn_generate_button = gr.Button("🧬 Generate & Run ProteinMPNN", variant="primary")
                        with gr.Column():
                            pmpnn_parameters_output = gr.Textbox(
                                label="ProteinMPNN Parameters (AI Analysis)",
                                lines=10
                            )
                            pmpnn_code_output = gr.Code(
                                language="python",
                                label="ProteinMPNN Python Code"
                            )
                            pmpnn_explanation_output = gr.Textbox(  # New component for explanation
                                label="Parameter Explanation",
                                lines=5,
                                interactive=False
                            )
                            pmpnn_audio_output = gr.Audio(  # New component for audio output
                                label="Listen to Parameter Analysis",
                                type="filepath"
                            )

                    # Add execution results section
                    gr.Markdown("## Execution Results")
                    with gr.Row():
                        with gr.Column():
                            pmpnn_execution_output = gr.Textbox(
                                label="Execution Status",
                                lines=5,
                                interactive=False
                            )
                        with gr.Column():
                            pmpnn_fasta_output = gr.Textbox(
                                label="Generated Sequences (FASTA)",
                                lines=10,
                                interactive=False
                            )

                    # Update the chat submission to use the new handler
                    pmpnn_msg.submit(
                        proteinmpnn_chat_handler,
                        inputs=[
                            pmpnn_msg,
                            pmpnn_chatbot,
                            gr.State([]),
                            pmpnn_parameters_output,
                            pmpnn_code_output
                        ],
                        outputs=[
                            pmpnn_msg,
                            pmpnn_chatbot,
                            gr.State([]),
                            pmpnn_audio_output
                        ]
                    )

                    # Update voice selection to use the new handler
                    pmpnn_voice_selector.change(
                        update_proteinmpnn_voice_handler,
                        inputs=[pmpnn_voice_selector],
                        outputs=[pmpnn_audio_output]
                    )

                    # Connect the components with the updated function
                    pmpnn_generate_button.click(
                        proteinmpnn_app,
                        inputs=[pmpnn_file_input, pmpnn_api_key],
                        outputs=[
                            pmpnn_parameters_output,
                            pmpnn_code_output,
                            pmpnn_explanation_output,
                            pmpnn_audio_output,
                            pmpnn_execution_output,
                            pmpnn_fasta_output
                        ]
                    )

                with gr.TabItem("AlphaFold2- Protein Design Step 3"):
                    gr.Markdown("""
                    ## AlphaFold2: Predict Protein Structures https://build.nvidia.com/deepmind/alphafold2

                    ### Model Overview
                    AlphaFold2 is DeepMind's revolutionary protein structure prediction system, achieving unprecedented accuracy in predicting 3D protein structures from amino acid sequences. The NVIDIA NGC platform provides an optimized implementation of AlphaFold2.

                    ### Key Parameters
                    - **algorithm**: MSA search algorithm (jackhmmer/hhblits)
                    - **e_value**: E-value threshold for sequence inclusion
                    - **iterations**: Number of search iterations
                    - **databases**: Sequence databases to search
                    - **relax_prediction**: Whether to relax the final structure
                    - **structure_model_preset**: Modeling preset (monomer/monomer_casp14/monomer_ptm/multimer)

                    ### Hot to Use
                    1. Take the amino acid sequence from your ProteinMPNN output or another sequence using only standard one-letter amino acid codes and enter it in the Amino Acid Sequence window.
                    2. Click "Generate AlphaFold2 Script" to create a customized Python script, the AI assistant will analyze your sequence and suggest optimal parameters for a successful NIM API call.
                    3. Copy the generated Python script and set up your environment with required dependencies (pip install requests)
                    4. Use the following URL to generate your AF2 API key under the Python tab: https://build.nvidia.com/deepmind/alphafold2?snippet_tab=Python
                    5. Run your Python script locally and enter your API key when prompted
                    6. If you encounter issues, you can also follow this link to the AlphaFold3 server when you can enter your sequence to complete the protein strucutre prediction from your sequence: https://alphafoldserver.com/

                    """)

                    # Chat interface with voice components
                    with gr.Row():
                        with gr.Column(scale=4):
                            af2_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False
                            )
                            af2_msg = gr.Textbox(
                                placeholder="Ask about AlphaFold2 or your protein sequence...",
                                show_label=False,
                                container=False
                            )
                            af2_clear = gr.ClearButton([af2_msg, af2_chatbot])

                        with gr.Column(scale=1):
                            af2_voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                container=True
                            )
                            af2_audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath"
                            )

                        def chat_with_alphafold2(message, chat_history, history_state):
                            """
                            Enhanced AlphaFold2 chat assistant with proper flow control and error handling
                            """
                            try:
                                if history_state is None:
                                    history_state = []

                                logger.info("Starting AlphaFold2 chat interaction")

                                # Phase 1: Query Preparation
                                try:
                                    logger.info("Preparing query and context")
                                    # Create a synchronous query to prevent stalling
                                    query_result = query_engine.query(
                                        message,
                                        response_mode="compact",  # Use compact mode for faster processing
                                        timeout=30  # Add timeout to prevent hanging
                                    )
                                    relevant_context = str(query_result.response)
                                    logger.info("Context retrieved successfully")
                                except Exception as e:
                                    logger.error(f"Error in query phase: {str(e)}")
                                    relevant_context = "Unable to retrieve context. Proceeding with basic response."

                                # Phase 2: Chat Completion
                                try:
                                    logger.info("Initiating chat completion")
                                    system_message = {
                                        "role": "system",
                                        "content": f"""You are an AI assistant specializing in AlphaFold2 and protein structure prediction.
                                        Use this relevant information to inform your response: {relevant_context}

                                        Help users with:
                                        1. Understanding AlphaFold2 capabilities and limitations
                                        2. Optimizing parameters for different protein types
                                        3. Interpreting results and confidence scores
                                        4. Best practices for structure prediction
                                        5. Integration with NVIDIA GPUs and optimization

                                        If you don't have enough information to answer a specific question,
                                        please say so and offer to help with other aspects of protein structure prediction."""
                                    }

                                    messages = [system_message] + history_state + [{"role": "user", "content": message}]

                                    response = openai_client.chat.completions.create(
                                        model="gpt-4o",
                                        messages=messages,
                                        max_tokens=16384,
                                        temperature=0,
                                        stream=False,
                                        timeout=60  # Add timeout for chat completion
                                    )

                                    response_text = response.choices[0].message.content
                                    logger.info("Chat completion successful")
                                except Exception as e:
                                    logger.error(f"Error in chat completion: {str(e)}")
                                    return message, chat_history, history_state, None

                                # Phase 3: Audio Generation
                                try:
                                    logger.info("Starting audio generation")
                                    audio_handler = AudioResponseHandler(openai_client)
                                    audio_file_path = audio_handler.text_to_speech(response_text)

                                    if not audio_file_path:
                                        logger.warning("Audio generation returned no file path")
                                        raise ValueError("Audio generation failed")

                                    logger.info(f"Audio generated successfully: {audio_file_path}")
                                except Exception as e:
                                    logger.error(f"Error in audio generation: {str(e)}")
                                    audio_file_path = None
                                    response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

                                # Update conversation state
                                chat_history = chat_history + [(message, response_text)]
                                history_state.append({"role": "assistant", "content": response_text})

                                logger.info("AlphaFold2 chat interaction completed successfully")
                                return message, chat_history, history_state, audio_file_path

                            except Exception as e:
                                logger.error(f"Critical error in AlphaFold2 chat system: {str(e)}")
                                error_message = "I apologize, but I encountered an unexpected error. Please try again."
                                chat_history = chat_history + [(message, error_message)]
                                return message, chat_history, history_state, None

                        def alphafold2_chat_handler(message, chat_history, state, parameters=None, code=None):
                            """Chat handler with monitoring and tracing"""
                            try:
                                with tracing.start_as_current_span("alphafold2_chat_interaction") as chat_span:
                                    chat_span.set_attribute("message_length", len(message))
                                    result = chat_with_alphafold2(message, chat_history, state)
                                    chat_monitor.record_success()
                                    return result
                            except Exception as e:
                                chat_monitor.record_failure(e)
                                logger.error(f"AlphaFold2 chat handler error: {str(e)}")
                                raise gr.Error(f"Chat system error: {str(e)}")

                        def update_alphafold2_voice_handler(voice):
                            """Voice update handler with monitoring"""
                            try:
                                audio_handler = AudioResponseHandler(openai_client)
                                audio_handler.set_voice(voice)
                                chat_monitor.record_success()
                                return None
                            except Exception as e:
                                chat_monitor.record_failure(e)
                                logger.error(f"Voice update error: {str(e)}")
                                raise gr.Error(f"Voice update error: {str(e)}")

                    gr.Markdown("## Generate AlphaFold2 Python Script")
                    with gr.Row():
                        with gr.Column():
                            af2_sequence_input = gr.Textbox(
                                label="Amino Acid Sequence",
                                placeholder="Enter sequence using standard one-letter amino acid codes...",
                                lines=5
                            )
                            af2_generate_button = gr.Button(
                                "🧬 Generate AlphaFold2 Script",
                                variant="primary"
                            )
                        with gr.Column():
                            af2_parameters_output = gr.Textbox(
                                label="AlphaFold2 Parameters (AI Analysis)",
                                lines=10
                            )
                            af2_code_output = gr.Code(
                                language="python",
                                label="AlphaFold2 Python Code"
                            )

                    # Update the Gradio interface connections
                    af2_msg.submit(
                        alphafold2_chat_handler,
                        inputs=[
                            af2_msg,
                            af2_chatbot,
                            gr.State([]),
                            af2_parameters_output,
                            af2_code_output
                        ],
                        outputs=[
                            af2_msg,
                            af2_chatbot,
                            gr.State([]),
                            af2_audio_output
                        ]
                    )

                    af2_voice_selector.change(
                        update_alphafold2_voice_handler,
                        inputs=[af2_voice_selector],
                        outputs=[af2_audio_output]
                    )

                    af2_generate_button.click(
                        fn=alphafold2_app,
                        inputs=[af2_sequence_input],
                        outputs=[af2_parameters_output, af2_code_output]
                    )

                    # Add clear button functionality
                    af2_clear.click(
                        lambda: ([], [], [], None),
                        outputs=[af2_msg, af2_chatbot, gr.State([]), af2_audio_output]
                    )

                    {
                        'chatbot': af2_chatbot,
                        'msg': af2_msg,
                        'voice_selector': af2_voice_selector,
                        'audio_output': af2_audio_output,
                        'sequence_input': af2_sequence_input,
                        'parameters_output': af2_parameters_output,
                        'code_output': af2_code_output
                    }

                # Update the MolMIM tab section with voice support
                with gr.TabItem("MolMIM"):
                    gr.Markdown("""
                    ## MolMIM: Molecular Generation and Optimization https://build.nvidia.com/nvidia/molmim-generate

                    ### Model Overview
                    MolMIM (Molecular Mutual Information Machine) is a latent variable model developed by NVIDIA for generating and optimizing molecules. It uses a transformer architecture trained on a large-scale dataset of molecules in SMILES format.

                    ### Key Features
                    - Generates new molecules by sampling from the latent space around a seed molecule
                    - Performs optimization using the CMA-ES algorithm in the model's latent space
                    - Utilizes Mutual Information Machine (MIM) learning for informative and clustered latent codes

                    ### How to Use
                    1. Enter a SMILES string for your seed molecule
                    2. Adjust the parameters as needed
                    3. Generate a Python script to run MolMIM using the API
                    4. Use the chat assistant for any questions or clarifications
                    """)

                    # Add voice components for MolMIM chat
                    with gr.Row():
                        with gr.Column(scale=4):
                            molmim_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False
                            )
                            molmim_msg = gr.Textbox(
                                placeholder="Ask about MolMIM or the generated script...",
                                show_label=False,
                                container=False
                            )
                            molmim_clear = gr.ClearButton([molmim_msg, molmim_chatbot])

                        with gr.Column(scale=1):
                            molmim_voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                container=True
                            )
                            molmim_audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath"
                            )

                    def molmim_chat_with_gpt4o(message, chat_history, history_state):
                        """
                        Enhanced MolMIM chat assistant with proper flow control and error handling
                        """
                        try:
                            if history_state is None:
                                history_state = []

                            logger.info("Starting MolMIM chat interaction")

                            # Phase 1: Query Preparation
                            try:
                                logger.info("Preparing query and context")
                                # Create a synchronous query to prevent stalling
                                query_result = query_engine.query(
                                    message,
                                    response_mode="compact",  # Use compact mode for faster processing
                                    timeout=30  # Add timeout to prevent hanging
                                )
                                relevant_context = str(query_result.response)
                                logger.info("Context retrieved successfully")
                            except Exception as e:
                                logger.error(f"Error in query phase: {str(e)}")
                                relevant_context = "Unable to retrieve context. Proceeding with basic response."

                            # Phase 2: Chat Completion
                            try:
                                logger.info("Initiating chat completion")
                                system_message = {
                                    "role": "system",
                                    "content": f"""You are an AI assistant specializing in molecular design and NVIDIA MolMIM technology.
                                    Use this relevant information to inform your response: {relevant_context}

                                    Focus on:
                                    1. MolMIM capabilities and limitations
                                    2. Molecular generation and optimization strategies
                                    3. NVIDIA GPU acceleration features
                                    4. Best practices for using MolMIM
                                    5. Property optimization techniques

                                    If you don't have enough information to answer a specific question,
                                    please say so and offer to help with other aspects of molecular design or NVIDIA's capabilities."""
                                }

                                messages = [system_message] + history_state + [{"role": "user", "content": message}]

                                response = openai_client.chat.completions.create(
                                    model="gpt-4o",
                                    messages=messages,
                                    max_tokens=16384,
                                    temperature=0,
                                    stream=False,
                                    timeout=60  # Add timeout for chat completion
                                )

                                response_text = response.choices[0].message.content
                                logger.info("Chat completion successful")
                            except Exception as e:
                                logger.error(f"Error in chat completion: {str(e)}")
                                return message, chat_history, history_state, None

                            # Phase 3: Audio Generation
                            try:
                                logger.info("Starting audio generation")
                                audio_handler = AudioResponseHandler(openai_client)
                                audio_file_path = audio_handler.text_to_speech(response_text)

                                if not audio_file_path:
                                    logger.warning("Audio generation returned no file path")
                                    raise ValueError("Audio generation failed")

                                logger.info(f"Audio generated successfully: {audio_file_path}")
                            except Exception as e:
                                logger.error(f"Error in audio generation: {str(e)}")
                                audio_file_path = None
                                response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

                            # Update conversation state
                            chat_history = chat_history + [(message, response_text)]
                            history_state.append({"role": "assistant", "content": response_text})

                            logger.info("MolMIM chat interaction completed successfully")
                            return message, chat_history, history_state, audio_file_path

                        except Exception as e:
                            logger.error(f"Critical error in MolMIM chat system: {str(e)}")
                            error_message = "I apologize, but I encountered an unexpected error. Please try again."
                            chat_history = chat_history + [(message, error_message)]
                            return message, chat_history, history_state, None

                    # Add these handler functions with monitoring
                    def molmim_chat_handler(message, chat_history, state):
                        """Chat handler with monitoring and tracing"""
                        try:
                            with tracing.start_as_current_span("molmim_chat_interaction") as chat_span:
                                chat_span.set_attribute("message_length", len(message))
                                result = molmim_chat_with_gpt4o(message, chat_history, state)
                                chat_monitor.record_success()
                                return result
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"MolMIM chat handler error: {str(e)}")
                            raise gr.Error(f"Chat system error: {str(e)}")

                    def update_molmim_voice_handler(voice):
                        """Voice update handler with monitoring"""
                        try:
                            audio_handler = AudioResponseHandler(openai_client)
                            audio_handler.set_voice(voice)
                            chat_monitor.record_success()
                            return None
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Voice update error: {str(e)}")
                            raise gr.Error(f"Voice update error: {str(e)}")

                    gr.Markdown("## Create MolMIM Python Script")
                    with gr.Row():
                        with gr.Column():
                            molmim_smiles_input = gr.Textbox(label="SMILES String", placeholder="Enter SMILES string...")
                            molmim_generate_button = gr.Button("🧪 Generate MolMIM Script", variant="primary")
                        with gr.Column():
                            molmim_parameters_output = gr.Textbox(label="MolMIM Parameters (AI Analysis)", lines=10)
                            molmim_code_output = gr.Code(language="python", label="MolMIM Python Code")

                    # Update the Gradio interface connections
                    molmim_msg.submit(
                        molmim_chat_handler,
                        inputs=[
                            molmim_msg,
                            molmim_chatbot,
                            gr.State([])
                        ],
                        outputs=[
                            molmim_msg,
                            molmim_chatbot,
                            gr.State([]),
                            molmim_audio_output
                        ]
                    )

                    # Connect voice selection
                    molmim_voice_selector.change(
                        update_molmim_voice_handler,
                        inputs=[molmim_voice_selector],
                        outputs=[molmim_audio_output]
                    )

                    # Update the clear button functionality
                    molmim_clear.click(
                        lambda: ([], [], [], None),
                        outputs=[molmim_msg, molmim_chatbot, gr.State([]), molmim_audio_output]
                    )

                    # Connect MolMIM script generation
                    molmim_generate_button.click(
                        molmim_app,
                        inputs=[molmim_smiles_input],
                        outputs=[molmim_parameters_output, molmim_code_output]
                    )

                with gr.TabItem("DiffDock"):
                    gr.Markdown("""
                    ## DiffDock: Molecular Docking via Diffusion Models https://build.nvidia.com/mit/diffdock

                    ### Model Overview
                    DiffDock is a generative diffusion model for drug discovery in molecular blind docking. It consists of two key components:
                    - **Score Model**: Generates potential poses through reverse diffusion
                    - **Confidence Model**: Ranks and evaluates generated poses

                    Key Features:
                    - No binding pocket information required
                    - Flexible molecule positioning and orientation
                    - Trained on PLINDER dataset for improved accuracy
                    - Supports both commercial and non-commercial use

                    ### Model Architecture
                    - Type: Score-Based Diffusion Model (SBDM)
                    - Network: Graph Convolution Neural Network
                    - Parameters: 20M in Score model
                    - Components: Embedding, 6-layer graph convolution, output layer
                    """)

                    # Chat interface with voice components
                    with gr.Row():
                        with gr.Column(scale=4):
                            diffdock_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False
                            )
                            diffdock_msg = gr.Textbox(
                                placeholder="Ask about DiffDock or molecular docking...",
                                show_label=False,
                                container=False
                            )
                            diffdock_clear = gr.ClearButton([diffdock_msg, diffdock_chatbot])

                        with gr.Column(scale=1):
                            diffdock_voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                container=True
                            )
                            diffdock_audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath"
                            )

                    def diffdock_chat_with_voice(message, chat_history, history_state, parameters=None, code=None):
                        """
                        Enhanced DiffDock chat assistant with proper flow control and error handling
                        """
                        try:
                            if history_state is None:
                                history_state = []

                            logger.info("Starting DiffDock chat interaction")

                            # Phase 1: Query Preparation
                            try:
                                logger.info("Preparing query and context")
                                # Create a synchronous query to prevent stalling
                                query_result = query_engine.query(
                                    message,
                                    response_mode="compact",  # Use compact mode for faster processing
                                    timeout=30  # Add timeout to prevent hanging
                                )
                                relevant_context = str(query_result.response)
                                logger.info("Context retrieved successfully")
                            except Exception as e:
                                logger.error(f"Error in query phase: {str(e)}")
                                relevant_context = "Unable to retrieve context. Proceeding with basic response."

                            # Phase 2: Chat Completion
                            try:
                                logger.info("Initiating chat completion")
                                system_message = {
                                    "role": "system",
                                    "content": f"""You are an AI assistant specializing in molecular docking with DiffDock and NVIDIA technologies.
                                    Use this relevant information to inform your response: {relevant_context}

                                    Current Context:
                                    Parameters: {parameters if parameters else 'No parameters provided'}
                                    Code: {code if code else 'No code generated'}

                                    Focus on:
                                    1. DiffDock capabilities and limitations
                                    2. Parameter optimization for different types of molecules
                                    3. Integration with NVIDIA GPU acceleration
                                    4. Best practices for molecular docking
                                    5. Interpreting docking results

                                    If you don't have enough information to answer a specific question,
                                    please say so and offer to help with other aspects of molecular docking or NVIDIA's capabilities."""
                                }

                                messages = [system_message] + history_state + [{"role": "user", "content": message}]

                                response = openai_client.chat.completions.create(
                                    model="gpt-4o",
                                    messages=messages,
                                    max_tokens=16384,
                                    temperature=0,
                                    stream=False,
                                    timeout=60  # Add timeout for chat completion
                                )

                                response_text = response.choices[0].message.content
                                logger.info("Chat completion successful")
                            except Exception as e:
                                logger.error(f"Error in chat completion: {str(e)}")
                                return message, chat_history, history_state, None

                            # Phase 3: Audio Generation
                            try:
                                logger.info("Starting audio generation")
                                audio_handler = AudioResponseHandler(openai_client)
                                audio_file_path = audio_handler.text_to_speech(response_text)

                                if not audio_file_path:
                                    logger.warning("Audio generation returned no file path")
                                    raise ValueError("Audio generation failed")

                                logger.info(f"Audio generated successfully: {audio_file_path}")
                            except Exception as e:
                                logger.error(f"Error in audio generation: {str(e)}")
                                audio_file_path = None
                                response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

                            # Update conversation state
                            chat_history = chat_history + [(message, response_text)]
                            history_state.append({"role": "assistant", "content": response_text})

                            logger.info("DiffDock chat interaction completed successfully")
                            return message, chat_history, history_state, audio_file_path

                        except Exception as e:
                            logger.error(f"Critical error in DiffDock chat system: {str(e)}")
                            error_message = "I apologize, but I encountered an unexpected error. Please try again."
                            chat_history = chat_history + [(message, error_message)]
                            return message, chat_history, history_state, None

                    def diffdock_chat_handler(message, chat_history, state, parameters=None, code=None):
                        """Chat handler with monitoring and tracing"""
                        try:
                            with tracing.start_as_current_span("diffdock_chat_interaction") as chat_span:
                                chat_span.set_attribute("message_length", len(message))
                                result = diffdock_chat_with_voice(message, chat_history, state, parameters, code)
                                chat_monitor.record_success()
                                return result
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"DiffDock chat handler error: {str(e)}")
                            raise gr.Error(f"Chat system error: {str(e)}")

                    def update_diffdock_voice_handler(voice):
                        """Voice update handler with monitoring"""
                        try:
                            audio_handler = AudioResponseHandler(openai_client)
                            audio_handler.set_voice(voice)
                            chat_monitor.record_success()
                            return None
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Voice update error: {str(e)}")
                            raise gr.Error(f"Voice update error: {str(e)}")

                    gr.Markdown("## Generate and Execute DiffDock")
                    with gr.Row():
                        with gr.Column():
                            diffdock_protein_input = gr.File(
                                label="Upload Protein Structure (PDB)",
                                file_types=[".pdb"]
                            )
                            diffdock_ligand_input = gr.Textbox(
                                label="Ligand SMILES String",
                                placeholder="Enter SMILES string..."
                            )
                            diffdock_api_key = gr.Textbox(
                                label="NVIDIA API Key",
                                placeholder="Enter your NVIDIA API key...",
                                type="password"
                            )
                            diffdock_generate_button = gr.Button("🎯 Generate & Run DiffDock", variant="primary")

                        with gr.Column():
                            diffdock_parameters_output = gr.Textbox(
                                label="DiffDock Parameters (AI Analysis)",
                                lines=10
                            )
                            diffdock_code_output = gr.Code(
                                language="python",
                                label="DiffDock Python Code"
                            )

                    def run_diffdock(api_key, protein_file, ligand_smiles, parameters):
                        try:
                            logger.info("Starting DiffDock run")

                            if not api_key or not protein_file or not ligand_smiles:
                                return "Please provide all required inputs.", None

                            # Log input validation
                            logger.info(f"Validating inputs:"
                                      f"\nAPI key present: {bool(api_key)}"
                                      f"\nProtein file present: {bool(protein_file)}"
                                      f"\nLigand SMILES length: {len(ligand_smiles)}")

                            protein_content = process_pdb_file(protein_file)
                            if not protein_content:
                                return "Error: Could not read PDB file.", None

                            status, output_files, success = execute_diffdock_script(
                                api_key,
                                protein_content,
                                ligand_smiles,
                                parameters
                            )

                            # Log the results
                            logger.info(f"DiffDock run completed:"
                                      f"\nSuccess: {success}"
                                      f"\nOutput files: {output_files if output_files else 'None'}")

                            if success and output_files:
                                file_outputs = []
                                for name, path in output_files.items():
                                    if os.path.exists(path):
                                        file_outputs.append(path)
                                return status, file_outputs if file_outputs else None
                            else:
                                return status, None

                        except Exception as e:
                            logger.error(f"Error in run_diffdock: {str(e)}")
                            logger.error(f"Full traceback: {traceback.format_exc()}")
                            return f"Error: {str(e)}", None

                    # Execution results section
                    gr.Markdown("## Execution Results")
                    with gr.Row():
                        with gr.Column():
                            diffdock_execution_output = gr.Textbox(
                                label="Execution Status",
                                lines=5,
                                interactive=False
                            )
                        with gr.Column():
                            diffdock_poses_output = gr.File(
                                label="Download Docking Results",
                                file_count="multiple",
                                interactive=False
                            )

                    # Connect the components
                    diffdock_generate_button.click(
                        diffdock_app,
                        inputs=[
                            diffdock_protein_input,
                            diffdock_ligand_input
                        ],
                        outputs=[
                            diffdock_parameters_output,
                            diffdock_code_output
                        ]
                    )


                    # Update the Gradio interface connections
                    diffdock_msg.submit(
                        diffdock_chat_handler,
                        inputs=[
                            diffdock_msg,
                            diffdock_chatbot,
                            gr.State([]),
                            diffdock_parameters_output,
                            diffdock_code_output
                        ],
                        outputs=[
                            diffdock_msg,
                            diffdock_chatbot,
                            gr.State([]),
                            diffdock_audio_output
                        ]
                    )

                    # Connect voice selection
                    diffdock_voice_selector.change(
                        update_diffdock_voice_handler,
                        inputs=[diffdock_voice_selector],
                        outputs=[diffdock_audio_output]
                    )

                    # Update the clear button functionality
                    diffdock_clear.click(
                        lambda: ([], [], [], None),
                        outputs=[diffdock_msg, diffdock_chatbot, gr.State([]), diffdock_audio_output]
                    )

                    # Connect execution button
                    diffdock_generate_button.click(
                        run_diffdock,
                        inputs=[
                            diffdock_api_key,
                            diffdock_protein_input,
                            diffdock_ligand_input,
                            diffdock_parameters_output
                        ],
                        outputs=[
                            diffdock_execution_output,
                            diffdock_poses_output
                        ]
                    )

                with gr.TabItem("Molecular Dynamics with GROMACS"):
                    gr.Markdown("""
                    ## GROMACS Chat Assistant and Protocol Generator https://catalog.ngc.nvidia.com/orgs/hpc/containers/gromacs

                    ### Overview
                    This tool generates optimized GROMACS molecular dynamics simulation protocols tailored to your protein structure.
                    It leverages NVIDIA GPU acceleration and modern GROMACS features for maximum performance.

                    ### Features
                    - Automatic structure analysis
                    - GPU-optimized protocol generation
                    - DeepSeek-enhanced protocol optimization
                    - NVIDIA technology integration analysis
                    - Voice-enabled chat assistance
                    """)

                    # Add voice components for GROMACS chat
                    with gr.Row():
                        with gr.Column(scale=4):
                            gromacs_chatbot = gr.Chatbot(
                                height=400,
                                bubble_full_width=False,
                                show_label=False
                            )
                            gromacs_msg = gr.Textbox(
                                placeholder="Ask about GROMACS simulations or your protocol...",
                                show_label=False,
                                container=False
                            )
                            gromacs_clear = gr.ClearButton([gromacs_msg, gromacs_chatbot])

                        with gr.Column(scale=1):
                            gromacs_voice_selector = gr.Dropdown(
                                choices=["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                                value="alloy",
                                label="Select Voice",
                                container=True
                            )
                            gromacs_audio_output = gr.Audio(
                                label="AI Response Audio",
                                type="filepath"
                            )

                    def chat_with_gromacs(message, chat_history, history_state, protocol=None, benefits=None):
                        """
                        Enhanced GROMACS chat assistant with proper flow control and error handling
                        """
                        try:
                            if history_state is None:
                                history_state = []

                            logger.info("Starting GROMACS chat interaction")

                            # Phase 1: Query Preparation
                            try:
                                logger.info("Preparing query and context")
                                # Create a synchronous query to prevent stalling
                                query_result = query_engine.query(
                                    message,
                                    response_mode="compact",  # Use compact mode for faster processing
                                    timeout=30  # Add timeout to prevent hanging
                                )
                                relevant_context = str(query_result.response)
                                logger.info("Context retrieved successfully")
                            except Exception as e:
                                logger.error(f"Error in query phase: {str(e)}")
                                relevant_context = "Unable to retrieve context. Proceeding with basic response."

                            # Phase 2: Chat Completion
                            try:
                                logger.info("Initiating chat completion")
                                system_message = {
                                    "role": "system",
                                    "content": f"""You are an AI assistant specializing in GROMACS molecular dynamics simulations and NVIDIA technologies.
                                    Use this information to inform your response:

                                    Relevant Context: {relevant_context}
                                    Current Protocol: {protocol if protocol else 'No protocol loaded'}
                                    NVIDIA Benefits: {benefits if benefits else 'No benefits analysis loaded'}

                                    Focus on:
                                    1. GROMACS simulation setup and optimization
                                    2. NVIDIA GPU acceleration features
                                    3. Best practices for molecular dynamics
                                    4. Troubleshooting common issues
                                    5. Integration with other tools and workflows

                                    If you don't have enough information to answer a specific question,
                                    please say so and offer to help with other aspects of GROMACS or NVIDIA's capabilities."""
                                }

                                messages = [system_message] + history_state + [{"role": "user", "content": message}]

                                response = openai_client.chat.completions.create(
                                    model="gpt-4o",
                                    messages=messages,
                                    max_tokens=16384,
                                    temperature=0,
                                    stream=False,
                                    timeout=60  # Add timeout for chat completion
                                )

                                response_text = response.choices[0].message.content
                                logger.info("Chat completion successful")
                            except Exception as e:
                                logger.error(f"Error in chat completion: {str(e)}")
                                return message, chat_history, history_state, None

                            # Phase 3: Audio Generation
                            try:
                                logger.info("Starting audio generation")
                                audio_handler = AudioResponseHandler(openai_client)
                                audio_file_path = audio_handler.text_to_speech(response_text)

                                if not audio_file_path:
                                    logger.warning("Audio generation returned no file path")
                                    raise ValueError("Audio generation failed")

                                logger.info(f"Audio generated successfully: {audio_file_path}")
                            except Exception as e:
                                logger.error(f"Error in audio generation: {str(e)}")
                                audio_file_path = None
                                response_text += "\n\n(Note: Audio response generation failed. Text response is still available.)"

                            # Update conversation state
                            chat_history = chat_history + [(message, response_text)]
                            history_state.append({"role": "assistant", "content": response_text})

                            logger.info("GROMACS chat interaction completed successfully")
                            return message, chat_history, history_state, audio_file_path

                        except Exception as e:
                            logger.error(f"Critical error in GROMACS chat system: {str(e)}")
                            error_message = "I apologize, but I encountered an unexpected error. Please try again."
                            chat_history = chat_history + [(message, error_message)]
                            return message, chat_history, history_state, None

                    # Handler functions with monitoring
                    def gromacs_chat_handler(message, chat_history, state, protocol=None, benefits=None):
                        """Chat handler with monitoring and tracing"""
                        try:
                            with tracing.start_as_current_span("gromacs_chat_interaction") as chat_span:
                                chat_span.set_attribute("message_length", len(message))
                                result = chat_with_gromacs(message, chat_history, state, protocol, benefits)
                                chat_monitor.record_success()
                                return result
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"GROMACS chat handler error: {str(e)}")
                            raise gr.Error(f"Chat system error: {str(e)}")

                    def update_gromacs_voice_handler(voice):
                        """Voice update handler with monitoring"""
                        try:
                            audio_handler = AudioResponseHandler(openai_client)
                            audio_handler.set_voice(voice)
                            chat_monitor.record_success()
                            return None
                        except Exception as e:
                            chat_monitor.record_failure(e)
                            logger.error(f"Voice update error: {str(e)}")
                            raise gr.Error(f"Voice update error: {str(e)}")

                    gr.Markdown("""## GROMACS Protocol Generator""")
                    with gr.Row():
                        with gr.Column():
                            gromacs_file_input = gr.File(
                                label="Upload Protein Structure (PDB)",
                                file_types=[".pdb"]
                            )
                            gromacs_generate_button = gr.Button("🧬 Generate GROMACS Protocol", variant="primary")
                        with gr.Column():
                            gromacs_protocol_output = gr.Textbox(
                                label="GROMACS Protocol (Improved with DeepSeek)",
                                lines=10
                            )
                            gromacs_benefits_output = gr.Textbox(
                                label="Benefits of NVIDIA Technology in GROMACS Simulation",
                                lines=5
                            )

                    # Connect the protocol generator
                    gromacs_generate_button.click(
                        gromacs_interface,
                        inputs=[gromacs_file_input],
                        outputs=[gromacs_protocol_output, gromacs_benefits_output]
                    )

                    # Connect the chat interface
                    gromacs_msg.submit(
                        gromacs_chat_handler,
                        inputs=[
                            gromacs_msg,
                            gromacs_chatbot,
                            gr.State([]),
                            gromacs_protocol_output,
                            gromacs_benefits_output
                        ],
                        outputs=[
                            gromacs_msg,
                            gromacs_chatbot,
                            gr.State([]),
                            gromacs_audio_output
                        ]
                    )

                    # Connect voice selection
                    gromacs_voice_selector.change(
                        update_gromacs_voice_handler,
                        inputs=[gromacs_voice_selector],
                        outputs=[gromacs_audio_output]
                    )

            gr.Markdown("""
            ---
            Developed by Joseph Steward. For questions, please contact jsteward2930@gmail.com.
            """)

    if __name__ == "__main__":
        with tracing.start_as_current_span("gradio_launch") as span:
            iface.launch(share=True)

except Exception as e:
    logger.error(f"Failed to initialize application: {str(e)}")
    print(f"Failed to initialize application: {str(e)}")

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d3167e7c4c22c51a15.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
